<p style="color:#FFF; background:#520099; padding:12px; font-size:20px; font-style:italic; text-align:center">
<span style="width:49%; display:inline-block; text-align:left">ROMAN Karina & TISSANDIER Mathilde</span>
<span style="width:50%; display:inline-block; text-align:right">CMI ISI M1</span>
<span style="font-size:45px; font-style:normal"><b>PROJET</b></span><br>
<span style="width:49%; display:inline-block">Outils Informatiques pour le Traitement des Données</span>

<h2 style="padding:15px; color:#FFFF; background: #8a2be2 ;  text-align:center">RECHERCHE DE CORRELATIONS ENTRE DES VARIABLES SOCIAUX-ECONOMIQUES EN FRANCE ET DES EVENEMENTS MAJEURS MONDIAUX</h2>

<h4 style="padding:15px; color:#FFFF; background: #bf7de8">Nous avons codé sur deux environnemts différents : VS Code et Jupyter Notebook. Il semble y avoir des difficultés avec l'un des graphiques lorsque le code est exécuté dans l'environnement VS Code. Pour une expérience plus fluide et pour profiter pleinement du projet, il est recommandé d'utiliser Jupyter Notebook pour l'étude du code.</h4>

<h2 style="padding:15px; color:#FFFF; background:#520099">Les imports</h2>

In [2]:
from pymongo import MongoClient
from io import BytesIO
import requests
import sys
import dash
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase
import plotly.express as px
import plotly.graph_objects as go
from dash import dcc, html
from dash.dependencies import Input, Output
from ipywidgets import interact, widgets
from IPython.display import display
import geopandas as gpd
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="statsmodels")
warnings.simplefilter(action='ignore', category=FutureWarning)

<h2 style="padding:15px; color:#FFFF; background:#520099">Bases de données</h2>

Pour améliorer la lisibilité et la visualisation de nos données stockées dans une base MongoDB, nous recommandons l'installation de l'extension MongoDB Compass. Cela facilitera la navigation et l'exploration des données, offrant une expérience plus conviviale et intuitive pour la gestion de la base de données.
- [Base de données MongoDB](https://www.mongodb.com/fr-fr)
- [MongoDB Compass](https://www.mongodb.com/products/tools/compass)

Installation sur Windows 10/11 : [Tutoriel](https://www.youtube.com/watch?v=gB6WLkSrtJk)

Installation sur Mac OS : [Tutoriel](https://www.youtube.com/watch?v=MyIiM7z_j_Y)

<h4>Extraction des données :</h4>

Nous allons utiliser plusieurs bases de données pour notre projet disponibles sur les sites web suivantes : 

**OpenDataSoft** avec des API

[INSEE - Indices des prix à la consommation, correction des valeurs saisonnières, inflation sous-jacente](
https://public.opendatasoft.com/explore/dataset/economicref-france-indice-prix-consommation/table/?disjunctive.sous_type&disjunctive.population&disjunctive.zone&dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6ImVjb25vbWljcmVmLWZyYW5jZS1pbmRpY2UtcHJpeC1jb25zb21tYXRpb24iLCJvcHRpb25zIjp7ImRpc2p1bmN0aXZlLnNvdXNfdHlwZSI6dHJ1ZSwiZGlzanVuY3RpdmUucG9wdWxhdGlvbiI6dHJ1ZSwiZGlzanVuY3RpdmUuem9uZSI6dHJ1ZX19LCJjaGFydHMiOlt7ImFsaWduTW9udGgiOnRydWUsInR5cGUiOiJsaW5lIiwiZnVuYyI6IkFWRyIsInlBeGlzIjoidmFsZXVyIiwic2NpZW50aWZpY0Rpc3BsYXkiOnRydWUsImNvbG9yIjoiI0ZGNTE1QSJ9XSwieEF4aXMiOiJwZXJpb2RlIiwibWF4cG9pbnRzIjoiIiwidGltZXNjYWxlIjoieWVhciIsInNvcnQiOiIifV0sImRpc3BsYXlMZWdlbmQiOnRydWUsImFsaWduTW9udGgiOnRydWV9)

[Nombre de demandeurs d'emploi par département - France](https://public.opendatasoft.com/explore/dataset/labouref-france-departement-quarter-jobseeker/table/?disjunctive.category&disjunctive.age_groups&disjunctive.reg_name&disjunctive.dep_area_code&disjunctive.dep_name&sort=date)

**Statista**, fichier au format XLS

[France: Inflation rate from 1987 to 2028 - XLS : (prévisions pour notre dernière partie)](https://www.statista.com/statistics/270353/inflation-rate-in-france/)

**Open URSSAF** avec des API

[Base de données de l’URSSAF](https://open.urssaf.fr/explore/?sort=modified&refine.theme=Observatoire+%C3%A9conomique&refine.theme=Observatoire+%C3%A9co+%2F+Embauches)

**INSEE**, avec des fichiers au format CSV et XLS

[Evolution du salaire](https://www.insee.fr/fr/information/3128533?INDICATEUR=3280709&qserie=salaire%20)

[Indice prix à la consommation](https://www.insee.fr/fr/information/3128533?INDICATEUR=2238611+2238612+2238613+3280708) 

[Statistiques salaires, etc..](https://www.insee.fr/fr/statistiques?debut=0&theme=24&categorie=10)

[Revenus, pouvoir d’achat, consommation](https://www.insee.fr/fr/statistiques?debut=0&theme=81+26+82+83+19+80&categorie=10)

[L'essentiel sur le chômage](https://www.insee.fr/fr/statistiques/4805248#graphique-figure2_radio1)

**Données Géolocalisation**

[Donnes de géolocalisation pour les régions et les départements français](https://france-geojson.gregoiredavid.fr)

___

Nous avons chronométré la durée pour exécuter la totalité du code.

**Sur Jupyter Notebook** : environ 8 minutes

**Sur VS Code** : environ 9 minutes

____

<h4>Stockage des données :</h4>

Pour analyser et traiter des ensembles de données volumineux avec MongoDB, une structure flexible, offrant la capacité d'adaptation nécessaire pour gérer ces informations de manière efficace et évolutive.

**Initialisation de la connexion à la base de données MongoDB :**

In [3]:
hostname = 'localhost'
port = 27017  # Default MongoDB port
client = MongoClient(hostname, port)
db = client['projet_Outils_Infos']

## **OpenDataSoft** avec des API

**INSEE - Indices des prix à la consommation, correction des valeurs saisonnières, inflation sous-jacente**

In [4]:
common_URL = 'https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/'

La fonction suivante permet de récupèrer des données depuis une API à partir de l'URL spécifiée, les insèrer dans une collection MongoDB, et gèrer la pagination pour récupérer l'ensemble des données. Elle prend en paramètres la base de données, le nom de la collection, l'URL de l'API, et une limite optionnelle pour le nombre d'éléments à récupérer à chaque requête. La récursivité est utilisée pour continuer la récupération jusqu'à la fin des données disponibles.

In [5]:
def fetch_and_insert_data(db, collection_name, url, limit=100):
    collection = db[collection_name]
    offset = 0

    def fetch_data_and_insert(offset):
        payload = {'limit': limit, 'offset': offset}
        response = requests.get(url, params=payload)

        if response.status_code == 200:
            data = response.json()

            try:
                collection.insert_many(data['results'])
            except :
                pass

            if len(data['results']) < limit:
                return
            else:
                fetch_data_and_insert(offset + limit)

    fetch_data_and_insert(offset)

EFIPC est l'abréviation de "economicref-france-indice-prix-consommation"

Nous effectuons une double boucle sur les années (years) et les zones géographiques (zones). À chaque itération, le script construit une URL spécifique pour récupérer les données d'un indicateur économique (indice des prix à la consommation) pour une année et une zone données. Ensuite, il utilise la fonction `fetch_and_insert_data` pour récupérer ces données depuis l'API correspondante et les insérer dans une collection MongoDB nommée "EFIPC". Ainsi, le script permet de collecter des informations sur l'indice des prix à la consommation pour différentes années et zones géographiques.

In [ ]:
years = ["2017", "2018", "2019","2020","2021","2022","2023"]
zones = ["France", "Guadeloupe", "La réunion", "Martinique", "Guyane", "Mayotte"]

for year in years:
    for zone in zones:
        EFIPC_url = f"{common_URL}economicref-france-indice-prix-consommation/records?select=periode%2Cvaleur&refine=zone%3A\"{zone}\"&refine=periode%3A\"{year}\""
        fetch_and_insert_data(db, "EFIPC", EFIPC_url)

In [ ]:
EFIPC = db.EFIPC.find()
EFIPC = pd.DataFrame(EFIPC)
columns_to_drop = ['_id']
EFIPC = EFIPC.drop(columns=columns_to_drop, axis=1)
EFIPC.head()

**Nombre de demandeurs d'emploi par département et région - France**

Nous utilisons le même mécanisme pour récupérer les données sur le nombre de demandeurs d'emploi en France.

In [ ]:
years = ["2017", "2018", "2019","2020","2021","2022","2023"]

for year in years:
    emploi_url = f"{common_URL}labouref-france-departement-quarter-jobseeker/records?select=quarter%2Creg_name%2Cdep_name%2Cnb_jobseeker&refine=quarter%3A\"{year}\""
    fetch_and_insert_data(db, "emploi", emploi_url)

Ensuite, nous récupérons les données de la collection MongoDB "emploi" et les stockons dans une variable appelée `emploi`. Nous convertissons les données MongoDB, initialement sous forme de dictionnaires, en un DataFrame pandas pour faciliter la manipulation. En définissant une liste de noms de colonnes à supprimer, nous éliminons la colonne `_id` du DataFrame `emploi`. Les colonnes du DataFrame sont renommées pour les rendre plus explicites. En utilisant la méthode split, nous divisons la colonne "Année" en deux parties (avant et après le tiret) et extrayons la deuxième partie dans une nouvelle colonne `Trimestres_année`. Nous extrayons la première partie de la colonne "Année" et la réassignons à la colonne "Année". Enfin, nous réorganisons l'ordre des colonnes dans le DataFrame pour placer `Année` en premier, suivi de `Trimestres_année`, `Demandeur_emploi`, `Région`et `Départemen`. Les premières lignes du DataFrame après ces transformations sont ensuite affichées.

In [ ]:
emploi = db.emploi.find()
emploi = pd.DataFrame(emploi)
columns_to_drop = ['_id']
emploi = emploi.drop(columns=columns_to_drop, axis=1)
emploi = emploi.rename(columns={'quarter':'Année', 'nb_jobseeker':'Demandeur_emploi','reg_name':'Région','dep_name':'Département'})
emploi['Trimestres_année'] = emploi['Année'].str.split('-', expand=True)[1]
emploi['Année'] = emploi['Année'].str.split('-', expand=True)[0]
emploi = emploi[['Année', 'Trimestres_année', 'Demandeur_emploi', 'Région', 'Département']]
emploi.head()

Nous souhaitons remplacer les données existantes dans la collection 'emploi' par cette nouvelle version. Il faudra donc supprimer les données existantes et ajouter la nouvelle version des données.

In [ ]:
emploi_dict_list = emploi.to_dict(orient='records')
db.emploi.delete_many({})
original_stdout = sys.stdout
sys.stdout = None
db.emploi.insert_many(emploi_dict_list, ordered=False)
sys.stdout = original_stdout

___

## **Statista, fichier au format XLS**

Inflation par an en pourcentage en France

Nous allons extraire ces données à partir d'un lien URL.

In [ ]:
url_inflation_rate = 'https://docs.google.com/spreadsheets/d/1arnCOlKiCZxKwsMeKhUj5c_0uWksqer_/export?format=xlsx'
inflation_rate = pd.read_excel(url_inflation_rate)
inflation_rate.head()

Drop les colonnes inutiles

In [ ]:
inflation_rate = inflation_rate.drop(columns={'Unnamed: 0','Unnamed: 3'},axis=1)
inflation_rate = inflation_rate.drop(index=0)
inflation_rate = inflation_rate.rename(columns={'Inflation rate in France 2028':'Année', 'Unnamed: 2':"taux_d'inflation"})
inflation_rate.head() 

Le code effectue plusieurs opérations sur le DataFrame ```inflation_rate```. Tout d'abord, il supprime les colonnes ```Unnamed: 0" et "Unnamed: 3``` afin de nettoyer le jeu de données. Ensuite, la première ligne du DataFrame est supprimée, ce qui peut être nécessaire pour éliminer des informations non pertinentes ou des en-têtes incorrects. Les colonnes sont ensuite renommées pour améliorer leur clarté : ```Inflation rate in France 2028``` devient ```Année``` et ```Unnamed: 2``` est renommée en ```taux_d'inflation```. Enfin, le résultat de ces transformations est affiché, montrant les premières lignes du DataFrame modifié. Ces étapes visent à préparer les données de manière plus lisible et significative.

In [ ]:
inflation_rate['Année'] = inflation_rate['Année'].astype(str).str.replace('*', '').astype(int)
inflation = inflation_rate[(inflation_rate['Année'] >= 2017) & (inflation_rate['Année'] <= 2022)]
inflation

On convertit la colonne 'Année' en chaîne de caractères, on retire les étoiles ('') le cas échéant, puis on convertit le résultat en entier. Cette séquence d'opérations garantit que la colonne 'Année' ne contient que des valeurs numériques. Ensuite, les données sont filtrées pour inclure uniquement les lignes où l'année est comprise entre 2017 et 2022 (inclus). Le résultat de cette opération crée un nouveau DataFrame appelé "inflation", qui contient exclusivement les données pour cette plage d'années. Ce processus contribue à restreindre l'analyse aux années spécifiées, facilitant ainsi une exploration plus ciblée des données.

Ensuite on filtre pour inclure uniquement celles dont l'année est comprise entre 2023 et 2028.

In [ ]:
inflation_prediction = inflation_rate[(inflation_rate['Année'] >= 2023) & (inflation_rate['Année'] <= 2028)]
inflation_prediction

On stocke ces données dans deux collections distinctes ```inflation```et ```inflation_prediction```dans la base de données.

In [ ]:
documents = inflation.to_dict(orient='records')
collection = db['inflation']
collection.insert_many(documents);

In [ ]:
documents = inflation_prediction.to_dict(orient='records')
collection = db['inflation_prediction']
collection.insert_many(documents);

___

## **Open URSSAF avec des API**

**Déclarations préalables à l'embauche mensuelles de plus d'un mois, par région**

In [ ]:
common_URL = 'https://open.urssaf.fr/api/explore/v2.1/catalog/datasets/'

DPALE_reg_grandsect est l'abréviation de "Déclarations préalables à l'embauche par région"

In [ ]:
years = ["2017", "2018", "2019","2020","2021","2022","2023"]

for year in years:
    DPALE_reg_url = f"{common_URL}dpae-par-departement-x-grand-secteur/records?select=annee%2C%20region%2C%20trimestre%2C%20nature_de_contrat%2C%20duree_de_contrat%2C%20dpae_cvs&refine=annee%3A%22{year}%22"
    fetch_and_insert_data(db, "DPALE_reg", DPALE_reg_url)

On supprime les colonnes "Unnamed: 0" et "Unnamed: 3" du DataFrame "inflation_rate". On supprime la première ligne du DataFrame "inflation_rate". Cela peut être utile si la première ligne contient des informations non pertinentes ou s'il s'agit d'un en-tête incorrect. On renomme les colonnes pour les rendre plus explicites. La colonne "Inflation rate in France 2028" est renommée en "Année" et "Unnamed: 2" est renommée en "taux_d'inflation". Finalement, on affiche les premières lignes du DataFrame "inflation_rate" après les transformations. 

In [ ]:
DPALE_reg = db.DPALE_reg.find()
DPALE_reg = pd.DataFrame(DPALE_reg)
DPALE_reg = DPALE_reg.drop('_id',axis=1)
DPALE_reg = DPALE_reg.rename(columns={'region':'Region', 'trimestre':'Trimestre','duree_de_contrat':'Duree_de_contrat','dpae_cvs':'Dpae_cvs'})
DPALE_reg.head()

In [ ]:
DPALE_reg_url_dict_list = DPALE_reg.to_dict(orient='records')
db.DPALE_reg.delete_many({})
original_stdout = sys.stdout
sys.stdout = None
db.DPALE_reg.insert_many(DPALE_reg_url_dict_list, ordered=False)
sys.stdout = original_stdout

**Déclarations préalables à l'embauche par an, par sexe x âge**

DPALE_sexe_age est l'abréviation de "Déclarations préalables à l'embauche par an, par sexe x âge"

De la même façon que précédemment pour les données issues des API, nous utilisons la fonction fetch_and_insert_data :

In [ ]:
years = ["2017", "2018", "2019","2020","2021","2022","2023"]

for year in years:
    DPALE_sexe_age_url = f"{common_URL}dpae-annuelles-sexe-age/records?select=annee%2C%20libelle_de_l_age%2C%20libelle_du_sexe%2C%20dpae_brut&refine=annee%3A%22{year}%22"
    fetch_and_insert_data(db, "DPALE_sexe_age", DPALE_sexe_age_url)

Par la suite, nous récupérons les données de la collection MongoDB `DPALE_reg` et les stockons dans une variable nommée `DPALE_reg`. Ensuite, nous convertissons les données MongoDB, initialement sous forme de dictionnaires, en un DataFrame pandas. Nous supprimons la colonne `_id` du DataFrame `DPALE_reg`. Pour rendre les noms des colonnes plus explicites, nous les renommons, modifiant ainsi les noms des colonnes `region`, `trimestre`, `duree_de_contrat` et `dpae_cvs` respectivement en `Region`, `Trimestre`, `Duree_de_contrat` et `Dpae_cvs`.

In [ ]:
DPALE_sexe_age = db.DPALE_sexe_age.find()
DPALE_sexe_age = pd.DataFrame(DPALE_sexe_age)
DPALE_sexe_age = DPALE_sexe_age.drop('_id',axis=1)
DPALE_sexe_age = DPALE_sexe_age.rename(columns={'annee':'Année', 'libelle_de_l_age':'Libelle_de_l_age','libelle_du_sexe':'Libelle_du_sexe','dpae_brut':'Dpae_brut'})
DPALE_sexe_age.head()

Nous supprimons à nouveau les données existantes afin d'introduire la nouvelle version des données dans le format souhaité.

In [ ]:
DPALE_sexe_age_dict_list = DPALE_sexe_age.to_dict(orient='records')
db.DPALE_sexe_age.delete_many({})
original_stdout = sys.stdout
sys.stdout = None
db.DPALE_sexe_age.insert_many(DPALE_sexe_age_dict_list, ordered=False)
sys.stdout = original_stdout

___

## **INSEE**, avec des fichiers au format CSV et XLS

Fonction qui télécharge un fichier CSV en utilisant une URL, applique des transformations sur les données, renomme les colonnes selon une liste donnée, exclut la colonne commune à tous les DataFrames ('Codes'), insère les données dans une collection MongoDB spécifiée, et retourne le DataFrame résultant : 

In [ ]:
def process_and_upload_data(file_id, new_column_names, dataframe_name):
    file_url = f'https://drive.google.com/uc?id={file_id}' # URL du fichier csv
    response = requests.get(file_url) # Téléchargement du fichier
    data = BytesIO(response.content)
    df = pd.read_csv(data, sep=';')
    df = df.drop(index=[0, 1, 2])
    df = df.drop(columns=['Codes'], axis=1)
    df = df.reset_index(drop=True)
    column_mapping = {old_name: new_name for old_name, new_name in zip(df.columns, new_column_names)}
    df = df.rename(columns=column_mapping)
    documents = df.to_dict(orient='records')
    collection = db[dataframe_name]
    collection.insert_many(documents)
    return df

### Evolution du salaire

evolution_salaire_h représente "L'Évolution du salaire net annuel moyen des postes à temps complet dans le secteur privé pour les hommes"

In [ ]:
file_id = '1NENgwf-BSl5PQXcy6DA-T6GjtGSUC7UH'
new_column_names = ['Année', 'Evolution_salaire']
dataframe_name = 'evolution_salaire_homme'
evolution_salaire_h = process_and_upload_data(file_id, new_column_names, dataframe_name)
evolution_salaire_h

evolution_salaire_f représente "L'Évolution du salaire net annuel moyen des postes à temps complet dans le secteur privé pour les femmes"

In [ ]:
file_id = '1EJPzxwq5pFBBz2aRsOoOP1JTr0oGkIbn'
new_column_names = ['Année', 'Evolution_salaire']
dataframe_name = 'evolution_salaire_femme'
evolution_salaire_f = process_and_upload_data(file_id, new_column_names, dataframe_name)
evolution_salaire_f.head()

evolution_salaire représente "Evolution des valeurs du salaire annuelles"

In [ ]:
file_id = '1ttPKWR75VJ4Ak88B_yoWh06w6ousTpWW'
new_column_names = ['Année', 'Evolution_salaire']
dataframe_name = 'evolution_salaire'
evolution_salaire = process_and_upload_data(file_id, new_column_names, dataframe_name)
evolution_salaire.head()

____

### Indice prix à la consommation

In [ ]:
file_id = '1hpUlChYuGxqAmX2IXVxt8HTykRL_zPW8'
new_column_names = ['Année-Trimestre', 'Indice_cout_travail']
dataframe_name = 'indice_cout'
indice_cout_travail = process_and_upload_data(file_id, new_column_names, dataframe_name)
indice_cout_travail['Année-Trimestre'] = indice_cout_travail['Année-Trimestre'].str.extract('(\d{4})')
indice_cout_travail = indice_cout_travail.rename(columns={'Année-Trimestre':'Année'})
indice_cout_travail.head()

Nous souhaitons modifier la base de données```indice_cout_travail``` et insérer cette nouvelle version. Pour ce faire, il faut supprimer les données existantes dans la base et y ajouter la nouvelle version.

In [ ]:
documents = indice_cout_travail.to_dict(orient='records')
db.indice_cout.delete_many({})
original_stdout = sys.stdout
sys.stdout = None
db.indice_cout.insert_many(documents, ordered=False)
sys.stdout = original_stdout

___

### Consommation

**Consommation Alimentaire**

Consommation mensuelle des ménages en biens - Alimentaire hors tabac - Volumes aux prix de l'année précédente, chaînés depuis 2014

In [ ]:
file_id = '16ExH-HhuX4YqqPTvWQRF3bpJgu8Q74vj'
new_column_names = ['Année-Mois', 'Consommation_Alimentaire']
dataframe_name = 'alimentation_consommation'
alimentaire_conso = process_and_upload_data(file_id, new_column_names, dataframe_name)
alimentaire_conso.head()

**Consommation mensuelle des ménages en biens - Énergie, eau, déchets, cokéfaction et raffinage**

Consommation mensuelle des ménages en biens - Énergie, eau, déchets, cokéfaction et raffinage - Volumes aux prix de l'année précédente, chaînés depuis 2014

In [ ]:
file_id = '1kLiF9MjiObiL_XZ-6k7f671guChadCvI'
new_column_names = ['Année-Mois', 'Consommation_Energie']
dataframe_name = 'energie_consommation'
energie_conso = process_and_upload_data(file_id, new_column_names, dataframe_name)
energie_conso.head()

___

### L'essentiel sur le chômage

Nous allons modifier manuelle les donnes disponibles sur le chomage (au lieu d'utiliser la fonction) car cet données ne sont organises de la meme fcon que les CVS, ces donnes se trouvent dans un fichier XLSX. 

Le code récupère des données sur le taux de chômage depuis une URL, effectue des transformations sur ces données pour les adapter, notamment en extrayant l'année et le trimestre à partir de la colonne "Trimestre". Ensuite, il sélectionne les colonnes pertinentes et filtre les données à partir de l'année 2017.

In [ ]:
url_chomage_datas = 'https://docs.google.com/spreadsheets/d/1XYayObiBGyPAhsMtWT45fHkuGH3oVVJJ/export?format=xlsx'
chomage = pd.read_excel(url_chomage_datas, skiprows=3)
chomage = chomage.drop(chomage.columns[2], axis=1)
chomage = chomage.iloc[:-4]
chomage['Trimestre_année'] = chomage['Trimestre'].str.split('-', expand=True)[1]
chomage['Année'] = chomage['Trimestre'].str.extract('(\d{4})')
chomage = chomage.drop('Trimestre', axis=1)
chomage = chomage[['Année', 'Trimestre_année', 'France (hors Mayotte)']]
chomage = chomage.reset_index(drop=True)
chomage['Année'] = pd.to_numeric(chomage['Année'])
chomage = chomage[::-1]
chomage = chomage[chomage['Année'] >= 2017]
chomage = chomage.reset_index(drop=True)
chomage.head()

Le code extrait les données du DataFrame chomage et les convertit en une liste de dictionnaires, où chaque dictionnaire représente une ligne du DataFrame. On ensere ces données dans une collection mongodb qui porte le nom 'chomage'.

In [ ]:
documents = chomage.to_dict(orient='records')
collection = db['chomage']
collection.insert_many(documents);

**Le chomage par département**

Pour les données du taux de chômage par département en fonction du trimestre et de l'année, nous téléchargeons les données dans un premier temps et les traitons à partir du lien URL en utilisant la fonction ```process_and_upload_data```. Nous effectuons des opérations de nettoyage, de renommage et de conversion des colonnes. Nous extrayons l'année et le trimestre, filtrons les données à partir de 2017, puis réorganisons les colonnes.

In [ ]:
file_id = '1kKl9YsJqhu3iCgWF2ZV4IbDRgEWg_ZIB'
new_column_names = ['Année']
dataframe_name = 'taux_chomage_departements'
taux_chomage_departements = process_and_upload_data(file_id, new_column_names, dataframe_name)
taux_chomage_departements = taux_chomage_departements.filter(regex='^(?!.*Codes)')
taux_chomage_departements.columns = taux_chomage_departements.columns.to_series().apply(lambda x: x.split('-', 1)[-1].strip().replace(" ", "_"))
taux_chomage_departements[['Année', 'Trimestre']] = taux_chomage_departements['Année'].str.extract(r'(\d{4})-(T\d)')
colonnes_a_convertir = [colonne for colonne in taux_chomage_departements.columns if colonne != 'Trimestre' and colonne != 'Année']
taux_chomage_departements[colonnes_a_convertir] = taux_chomage_departements[colonnes_a_convertir].astype(float)
taux_chomage_departements['Année'] = taux_chomage_departements['Année'].astype(int)
taux_chomage_departements = taux_chomage_departements[taux_chomage_departements['Année'] > 2016]
nouvel_ordre_colonnes = ['Année', 'Trimestre'] + [col for col in taux_chomage_departements.columns if col not in ['Année', 'Trimestre']]
taux_chomage_departements = taux_chomage_departements[nouvel_ordre_colonnes]
taux_chomage_departements.head()

Nous souhaitons modifier la base de données ```taux_chomage_departements``` et insérer cette nouvelle version obtenue apres le traitement de ces données. Pour ce faire, il faut supprimer les données existantes dans la base et y ajouter la nouvelle version.

In [ ]:
documents = taux_chomage_departements.to_dict(orient='records')
db.taux_chomage_departements.delete_many({})
original_stdout = sys.stdout
sys.stdout = None
db.taux_chomage_departements.insert_many(documents, ordered=False)
sys.stdout = original_stdout

## **Données Géolocalisation**

Nous allons récupérer les données de géolocalisation sans nécessité de traitement ni de modification.

Pour les régions françaises : 

In [ ]:
url = 'https://drive.google.com/uc?id=1_bKCQYlMiU-lJMgLvFOvfBqgHH2IQD_p&export=download'
régions = gpd.read_file(url)
régions = régions.drop('code',axis=1)
régions.head()

Pour les départements français :

In [ ]:
url = 'https://drive.google.com/uc?id=1evvclIka5gCZrWnH4A71gOURBjHJ8fhI'
départements = gpd.read_file(url)
départements = départements.drop('code',axis=1)
départements.head()

Nous ne pouvons pas ajouter dans une base de données les données de type "POLYGON".

___

## Synthèse

Nous disposons au total de 14 bases de données, comprenant 5 issues d'API, une extraite d'un fichier XLSX, et 8 provenant de fichiers CSV. Ces bases de données regroupent une variété d'informations, notamment :
- **EFIPC** : Indices des prix à la consommation, correction des valeurs saisonnières, inflation sous-jacente (API) 
- **emploi** : Nombre de demandeurs d'emploi par département en France (API) 
- **inflation** : Inflation annuelle en pourcentage en France entre 2017 et 2023 (XLSX) 
- **inflation_prediction** : Prédiction sur l'inflation annuelle en pourcentage en France entre 2017 et 2023 (XLSX)
- **DPALE_reg** : Déclarations préalables à l'embauche par département dans les grands secteurs (API) 
- **DPALE_sexe_age** : Déclarations préalables à l'embauche par an, par sexe et âge (API) 
- **evolution_salaire_homme** : Évolution du salaire net annuel moyen des postes à temps complet dans le secteur privé pour les hommes (CSV) 
- **evolution_salaire_femme** : Évolution du salaire net annuel moyen des postes à temps complet dans le secteur privé pour les femmes (CSV)
- **evolution_salaire** : Évolution des valeurs du salaire annuel (CSV) 
- **indice_cout** : Indice du coût du travail - Coût horaire - Ensemble des secteurs (CSV) 
- **alimentaire_consommation** : Consommation mensuelle des ménages en biens - Alimentaire hors tabac - Volumes aux prix de l'année précédente, chaînés depuis 2014 (CSV) 
- **energie_conso** : Consommation mensuelle des ménages en biens - Énergie, eau, déchets, cokéfaction et raffinage - Volumes aux prix de l'année précédente, chaînés depuis 2014 (CSV) 
- **chomage** : Evolution moyennne du chômage entre 2017 et 2023 (XLXS)
- **taux_chomage_departements** : Evolution du taux de chômage par département de 2017 à 2023 (CSV)
- **régions** : Donnés de géolocalisation pour les régions françaises (GEOJSON)
- **départements** : Donnés de géolocalisation pour les départements français (GEOJSON)

<h2 style="padding:15px; color:#FFFF; background:#520099">Analyse</h2>

Dans cette partie, on a décidé de faire commencer les impacts de la pandémie de covid 19 à partir de 2020, on a considéré que les effets n'étaient visibles qu'à partir de 2020 et non au cours de l'année 2019.

<h2 style="padding:15px; color:#FFFF; background: #8a2be2">Partie 1 : Le salaire et le chômage</h2>

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">L'evolution du salaire</h2>

Étudier l'évolution des salaires entre hommes et femmes, ainsi que le salaire moyen, est important pour plusieurs raisons, notamment :

1. **Équité salariale :** L'écart de rémunération entre hommes et femmes est une préoccupation majeure en matière d'équité salariale. L'étude de l'évolution des salaires permet de détecter et de comprendre les disparités salariales entre les genres, contribuant ainsi à promouvoir des pratiques salariales plus équitables.

2. **Justice sociale :** En comprenant l'évolution des salaires, les politiques et les pratiques qui ont contribué à l'écart salarial entre hommes et femmes peuvent être identifiées. Cela peut conduire à des initiatives visant à corriger les inégalités et à promouvoir la justice sociale.

3. **Productivité et engagement :** Un traitement équitable des employés, indépendamment de leur genre, peut améliorer la satisfaction au travail, l'engagement et la productivité. En étudiant l'évolution des salaires, les entreprises peuvent identifier des opportunités d'amélioration des pratiques de rémunération pour favoriser un environnement de travail équitable et inclusif.

4. **Conformité aux réglementations :** Dans de nombreux pays, des réglementations et des lois existent pour promouvoir l'équité salariale entre hommes et femmes. Étudier l'évolution des salaires permet aux entreprises de s'assurer qu'elles respectent les normes légales en matière d'égalité des salaires.

5. **Image de l'entreprise :** Les entreprises qui démontrent un engagement envers l'équité salariale peuvent améliorer leur image et leur réputation, attirant ainsi des talents diversifiés et fidélisant leurs employés.

6. **Analyse des tendances du marché du travail :** Comprendre l'évolution des salaires peut également aider à analyser les tendances du marché du travail. Par exemple, des disparités salariales persistantes peuvent signaler des problèmes structurels dans l'économie ou le marché du travail.

En somme, l'étude de l'évolution des salaires entre hommes et femmes, ainsi que le salaire moyen, est cruciale pour promouvoir l'équité, la justice sociale et la conformité aux normes légales, tout en favorisant des environnements de travail inclusifs et productifs.

On fusionne les trois DataFrames qui traintent de l'évolution du salaire sur la colonne "Année" :

Au tout début, nous récupérons les données sur l'évolution des salaires des hommes, des femmes et l'évolution globale des salaires à partir de trois collections MongoDB distinctes. Les données sont ensuite converties en DataFrames pandas, et les colonnes sont transformées en types numériques. Après fusion des DataFrames sur la colonne "Année", les résultats sont combinés en un seul DataFrame, éliminant les colonnes contenant des valeurs manquantes. Enfin, les premières lignes du DataFrame résultant, représentant l'évolution globale des salaires, sont affichées.

In [ ]:
evolution_salaire_homme = db.evolution_salaire_homme.find()
evolution_salaire_femme = db.evolution_salaire_femme.find()
evolution_salaire = db.evolution_salaire.find()

evolution_salaire_homme = pd.DataFrame(evolution_salaire_homme)
evolution_salaire_femme = pd.DataFrame(evolution_salaire_femme)
evolution_salaire = pd.DataFrame(evolution_salaire)

evolution_salaire_homme = evolution_salaire_homme.apply(pd.to_numeric, errors='coerce', downcast='integer')
evolution_salaire_femme = evolution_salaire_femme.apply(pd.to_numeric, errors='coerce', downcast='integer')
evolution_salaire = evolution_salaire.apply(pd.to_numeric, errors='coerce', downcast='integer')

evolution_salaire_df = pd.merge(evolution_salaire_homme, evolution_salaire_femme, on='Année', suffixes=('_homme', '_femme'))
evolution_salaire_df = pd.merge(evolution_salaire_df, evolution_salaire, on='Année', suffixes=('_merged', '_salaire'))
evolution_salaire_df = evolution_salaire_df.dropna(axis=1)
evolution_salaire_df.head()

In [ ]:
plt.plot(evolution_salaire_df['Année'], evolution_salaire_df['Evolution_salaire_homme'], label='Evolution_salaire_homme', marker='o')
plt.plot(evolution_salaire_df['Année'], evolution_salaire_df['Evolution_salaire_femme'], label='Evolution_salaire_femme', marker='o')
plt.plot(evolution_salaire_df['Année'], evolution_salaire_df['Evolution_salaire'], label='Evolution_salaire', marker='o')

plt.axvline(x=2020, color='r', linestyle='--', label='Pandémie Covid')
plt.axvline(x=2022, color='b', linestyle='--', label='Début Guerre Ukraine')

plt.title('Évolution des salaires par genre au fil des années')
plt.xlabel('Années')
plt.ylabel('Évolution des salaires')
plt.legend()
plt.show()

**Étude du graphique sur l'évolution des salaires par genre au fil des années**

Le graphique montre l'évolution des salaires moyens des hommes et des femmes en France entre 2017 et 2023. Les salaires sont mesurés en euros constants, c'est-à-dire qu'ils sont ajustés en fonction de l'inflation.

**Tendances générales**

Le graphique montre que les salaires des hommes et des femmes ont augmenté de manière significative au cours de la période étudiée. Cependant, l'écart de salaire entre les hommes et les femmes s'est réduit de manière constante.

**Pandémie de COVID-19**

La pandémie de COVID-19 a eu un impact négatif sur l'économie française, y compris sur les salaires. En 2020, les salaires moyens ont diminué de 5.4 % pour les hommes et de 5.5 % pour les femmes.

**Guerre en Ukraine**

La guerre en Ukraine a également eu un impact négatif sur l'économie française, y compris sur les salaires. En 2022, les salaires moyens ont augmenté de 0.5 % pour les hommes et de 1.9 % pour les femmes.

**Interprétations**

L'évolution des salaires par genre au fil des années peut être interprétée de différentes manières.

* **Une amélioration de la situation des femmes**

L'écart de salaire entre les hommes et les femmes s'est réduit de manière constante au cours de la période étudiée. Cela peut être interprété comme une amélioration de la situation des femmes sur le marché du travail.

* **Les effets de la pandémie et de la guerre**

La pandémie de COVID-19 et la guerre en Ukraine ont eu un impact négatif sur l'économie française, y compris sur les salaires. Cependant, l'impact de ces événements a été plus important pour les hommes que pour les femmes.

* **L'impact de la politique gouvernementale**

La politique gouvernementale peut avoir un impact sur les salaires. Par exemple, les politiques d'égalité salariale peuvent contribuer à réduire l'écart de salaire entre les hommes et les femmes.

**Conclusion**

L'évolution des salaires par genre au fil des années est un phénomène complexe qui peut être interprété de différentes manières. Il est important de tenir compte des facteurs économiques et politiques qui peuvent affecter les salaires.

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">Nombre de demandeurs d'emploi</h2>

Autout debut nous allons extraire les données sur le nombtre de chercheurs d'emploi qui se trouvent dans la collection emploi et les modifier de facon à pouvoir crer une visualisation intercative à l'aide de la fonctionalité Dash.

In [ ]:
emploi = db.emploi.find()
emploi = pd.DataFrame(emploi)
emploi = emploi.drop('_id',axis=1)

In [ ]:
emploi_copy = emploi.copy()
emploi_copy['Année'] = pd.to_datetime(emploi_copy['Année'])
emploi_copy['Année'] = emploi_copy['Année'].dt.year
regions_dropdown = [{'label': region, 'value': region} for region in emploi_copy['Région'].unique()]
regions_dropdown.append({'label': 'Synthèse', 'value': 'Synthèse'})

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='region-dropdown',
        options=regions_dropdown,
        value='Synthèse'
    ),
    dcc.Graph(id='employment-graph')
])

@app.callback(
    Output('employment-graph', 'figure'),
    [Input('region-dropdown', 'value')]
)
def update_plot(selected_region):
    if selected_region == 'Synthèse':
        aggregated_data = emploi_copy.groupby(['Année', 'Région'])['Demandeur_emploi'].mean().reset_index()
        fig = px.bar(aggregated_data, x='Année', y='Demandeur_emploi',
                     color='Région',
                     labels={'Demandeur_emploi': 'Nombre total de demandeurs d\'emploi'},
                     color_discrete_sequence=px.colors.qualitative.Pastel,
                     height=600, width=1000)
        fig.update_layout(title=f'Nombre total de demandeurs d\'emploi pour toutes les régions',
                          xaxis_title='Année', yaxis_title='Nombre total de demandeurs d\'emploi',
                          barmode='group')
        return fig
    else:
        filtered_data = emploi_copy[emploi_copy['Région'] == selected_region]
        aggregated_data = filtered_data.groupby(['Année', 'Trimestres_année'])['Demandeur_emploi'].mean().reset_index()
        fig = px.bar(aggregated_data, x='Année', y='Demandeur_emploi',
                     color='Trimestres_année',
                     color_discrete_map={'T1': '#DDA0DD', 'T2': '#00CCFF', 'T3': '#33CC99', 'T4': '#9999FF'},
                     labels={'Demandeur_emploi': 'Nombre total de demandeurs d\'emploi'},
                     category_orders={'Trimestres_année': ['T1', 'T2', 'T3', 'T4']},
                     height=600, width=1000)
        fig.update_layout(title=f'Nombre total de demandeurs d\'emploi pour la région {selected_region}',
                          xaxis_title='Année', yaxis_title='Nombre total de demandeurs d\'emploi',
                          barmode='group')  # Utiliser 'group' pour avoir des barres côte à côte
        return fig

if __name__ == '__main__':
    app.run_server(debug=True)

Le graphioque en barres représente le nombre total de demandeurs d'emploi pour différentes régions de France sur une période de 2017 à 2023. Chaque barre colorée correspond à une région spécifique, comme indiqué dans la légende à droite du graphique. Les années sont indiquées sur l'axe des abscisses (horizontalement) et le nombre de demandeurs d'emploi est indiqué sur l'axe des ordonnées (verticalement), avec des valeurs allant de 0 à 60 000.

**Voici quelques observations basées sur ce graphique :**
- La région Île-de-France semble avoir le plus grand nombre de demandeurs d'emploi chaque année, ce qui est représenté par la barre la plus haute dans chaque groupe d'années.
Certaines régions semblent avoir une tendance à la baisse ou à la stabilité en termes de nombre de demandeurs d'emploi au fil des ans.
- Il y a une variation notable dans le nombre de demandeurs d'emploi entre les régions, certaines comme la Corse et les départements d'outre-mer (par exemple, Guadeloupe et Martinique) ayant des nombres relativement plus faibles de demandeurs d'emploi.

Nous analysons l'impact de la pandémie de COVID-19 en 2020 et de la guerre en Ukraine commencée en 2022 sur le nombre de demandeurs d'emploi en France, comparons les données de ces années spécifiques avec les années précédentes et suivantes présentées dans le graphique.

**Pandémie de COVID-19 en 2020 :**

Il y a eu une augmentation généralisée du nombre de demandeurs d'emploi dans presque toutes les régions en 2020, ce qui peut être attribué aux effets économiques de la pandémie. Les confinements et les restrictions sanitaires ont fortement impacté de nombreux secteurs, entraînant des fermetures d'entreprises et des licenciements.

La région Île-de-France, qui est un centre économique important avec de nombreux secteurs d'activité, semble avoir été particulièrement touchée, avec un pic notable en 2020.

D'autres régions montrent également une augmentation du nombre de demandeurs d'emploi en 2020, mais avec des variations selon la structure économique de chaque région.

**Guerre en Ukraine en 2022 :**

En 2022, l'impact de la guerre en Ukraine sur le marché du travail français n'est pas immédiatement évident dans le graphique. Il pourrait y avoir plusieurs raisons à cela : l'impact indirect de la guerre pourrait être plus diffus, ou bien les effets pourraient ne pas être immédiatement visibles dans les chiffres de l'emploi.

Les conséquences économiques de la guerre, qui peuvent inclure l'augmentation des prix de l'énergie et des perturbations dans les chaînes d'approvisionnement, pourraient avoir un effet retardé sur l'emploi.

Les données pour 2022 ne montrent pas de pic significatif par rapport à 2021, ce qui suggère que soit l'impact sur le marché du travail n'a pas été aussi direct que celui de la pandémie, soit les mesures de soutien économique mises en place ont pu atténuer les effets sur l'emploi.

**Comparaison avec les autres années :**

En comparant 2020 et 2022 aux années précédentes et suivantes, on peut observer une tendance à l'augmentation en 2020 liée à la pandémie, suivie d'une certaine stabilisation ou diminution du nombre de demandeurs d'emploi en 2021 et 2023.
Il est important de noter que les données pour 2023 peuvent encore être provisoires et susceptibles de changer car nous n'avons toujours pas les données disponibles pour le Trimestre T4.

En utilisant la liste déroulante située au-dessus du graphique, nous pouvons sélectionner la région qui nous intéresse pour effectuer cette même analyse.

Nous pouvons allors realiser une analyse plus approfondie, en calculant le pourcentage d'augmentation entre la periode 2019 et 2020 du nombre total de demandeurs d'emploi et observer s'il existe une evolution majeure.

In [ ]:
def calculate_percentage_increase(emploi, start_year, end_year=None, start_quarter=None, end_quarter=None):
    emploi_copy = emploi.copy()
    emploi_copy['Année'] = pd.to_datetime(emploi_copy['Année'])
    emploi_copy['Année'] = emploi_copy['Année'].dt.year

    if end_year is not None:
        data_start = emploi_copy[emploi_copy['Année'] == start_year]
        data_end = emploi_copy[emploi_copy['Année'] == end_year]
    elif start_quarter and end_quarter:
        data_start = emploi_copy[(emploi_copy['Année'] == start_year) & (emploi_copy['Trimestres_année'] == start_quarter)]
        data_end = emploi_copy[(emploi_copy['Année'] == start_year) & (emploi_copy['Trimestres_année'] == end_quarter)]

    grouped_start = data_start.groupby('Région')['Demandeur_emploi'].sum()
    grouped_end = data_end.groupby('Région')['Demandeur_emploi'].sum()

    percentage_increase = ((grouped_end - grouped_start) / grouped_start.abs()) * 100

    result_df = pd.DataFrame({
        'Région': percentage_increase.index,
        f'Pourcentage_d\'augmentation_{start_year}_{end_year}' if end_year else f'Pourcentage_d\'augmentation_{start_quarter}_{end_quarter}_{start_year}': percentage_increase.values
    })

    return result_df.sort_values(by=f'Pourcentage_d\'augmentation_{start_year}_{end_year}' if end_year else f'Pourcentage_d\'augmentation_{start_quarter}_{end_quarter}_{start_year}', ascending=False).reset_index(drop=True)

In [ ]:
result_2019_2020 = calculate_percentage_increase(emploi, start_year=2019, end_year=2020)

In [ ]:
regions_a_supprimer = ['La Réunion', 'Guadeloupe', 'Guyane', 'Martinique']
result_2019_2020 = result_2019_2020[~result_2019_2020['Région'].isin(regions_a_supprimer)]
result_2019_2020

D'apres ce DataFrame, nous constatons que le nombre de demandeurs d'emploi en France a augmenté de 4 % entre 2019 et 2020. Cette augmentation est due à plusieurs facteurs, notamment la crise sanitaire de la COVID-19, qui a entraîné des fermetures d'entreprises et des licenciements.

L'augmentation du nombre de demandeurs d'emploi est plus marquée dans certaines régions que dans d'autres. L'Île-de-France, qui concentre la majorité des emplois en France, a enregistré une des plus forte augmentation, avec 4,7 %. Les Pays de la Loire (3,7 %), l'Occitanie (2,4 %), la Nouvelle-Aquitaine (2,1 %) et la Corse (8,4 %) ont également connu une augmentation significative.

En revanche, une région a enregistré une baisse du nombre de demandeurs d'emploi, c'est les Hauts-de-France. Cette région est moins dépendantes des secteurs économiques qui ont été les plus touchés par la crise sanitaire.

**Évolution majeure**

L'augmentation du nombre de demandeurs d'emploi en France est une évolution majeure, qui a des conséquences sociales et économiques importantes. Elle touche notamment les jeunes, les personnes peu qualifiées et les personnes en situation de handicap.

Le gouvernement français a mis en place plusieurs mesures pour faire face à cette crise, notamment le plan de relance économique et le dispositif d'activité partielle. Ces mesures ont permis de limiter l'ampleur de l'augmentation du chômage, mais elles ne suffisent pas à répondre à tous les besoins.

**Analyse plus approfondie sur 2020**

Nous souhaitons alors analyser encore plus en détail ces évolutions entre le premier trimestre T1 et le second T2 de l'année 2020, afin d'observer plus en profondeur l'impact possible de la pandémie sur le nombre de chercheurs d'emploi en France.

In [ ]:
result_2020_T1_T2 = calculate_percentage_increase(emploi, start_year=2020, start_quarter='T1', end_quarter='T2')
result_2020_T1_T2 = result_2020_T1_T2[~result_2020_T1_T2['Région'].isin(regions_a_supprimer)]
result_2020_T1_T2

Ce DataFrame contient les données sur l'évolution du nombre de demandeurs d'emploi en France entre le premier trimestre (T1) et le second trimestre (T2) de l'année 2020, classées par région.

Les variations régionales du nombre de demandeurs d'emploi sont significatives. La Corse, une région axée sur le tourisme, a enregistré la plus forte augmentation, avec une hausse de 28,6 %. D'autres régions ayant connu une forte augmentation sont les Pays de la Loire (7,3 %), Provence-Alpes-Côte d'Azur (7,2 %), la Bretagne (7 %) et l'Auvergne-Rhône-Alpes (6,7 %).

**Causalité**

L'augmentation du nombre de demandeurs d'emploi entre le premier trimestre (T1) et le second trimestre (T2) de l'année 2020 est attribuable à la crise sanitaire de la COVID-19. En effet, le confinement du printemps 2020 a mis à l'arrêt de nombreux secteurs économiques, notamment le tourisme, la restauration et les services. Cette situation a entraîné la perte de nombreux emplois, provoquant ainsi une augmentation du nombre de demandeurs d'emploi.

Les régions ayant enregistré une forte augmentation du nombre de demandeurs d'emploi sont celles qui concentrent la majorité des emplois et qui ont été particulièrement touchées par la crise sanitaire. Il s'agit notamment de l'Île-de-France, de la Nouvelle-Aquitaine, de l'Occitanie, des Pays de la Loire et de la Corse.

De la même façon, nous pouvons analyser afin de déterminer s'il existe un impact de la guerre en Ukraine, déclenchée au début de l'année 2022, sur le nombre de demandeurs d'emploi en France.

In [ ]:
result_2021_2022 = calculate_percentage_increase(emploi, start_year=2021, end_year=2022)
result_2021_2022 = result_2021_2022[~result_2021_2022['Région'].isin(regions_a_supprimer)]
result_2021_2022

Le DataFrame ci-dessus montre l'évolution du nombre de demandeurs d'emploi en France entre 2021 et 2022. On observe une baisse significative du nombre de demandeurs d'emploi entre ces deux années dans toutes les regions.

Cette baisse est due à plusieurs facteurs, notamment :

- La reprise économique, qui a entraîné la création d'emplois ;
- Les mesures de soutien à l'emploi mises en place par le gouvernement français, notamment le plan de relance économique
- L'amélioration de la situation sanitaire, qui a permis la réouverture des entreprises et des établissements publics.

**Impact de la guerre en Ukraine**

D'après les données disponibles, nous ne constatons pas d'impact réel de la guerre en Ukraine sur le nombre de demandeurs d'emploi. Ainsi, l'incidence de la guerre en Ukraine sur l'emploi en France semble limitée, comme observé précédemment sur le graphique en barres. En effet, la France dépend moins de l'énergie russe que d'autres pays européens. De plus, le gouvernement français a mis en place des mesures de soutien à l'économie, notamment des aides aux entreprises et des dispositifs de protection sociale. Pour observer d'éventuels impacts, il est nécessaire de se focaliser sur l'analyse d'autres aspects.

**Carte graphique par région**

Dans un dernier temps, nous pourrons observer ces résultats sur la carte de la France afin de les visualiser plus clairement.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(30, 8))

combined_data = pd.concat([result_2019_2020['Pourcentage_d\'augmentation_2019_2020'],
                           result_2020_T1_T2['Pourcentage_d\'augmentation_T1_T2_2020'],
                           result_2021_2022['Pourcentage_d\'augmentation_2021_2022']], axis=0)

vmin, vmax = combined_data.min(), combined_data.max()

merged_2019_2020 = régions.merge(result_2019_2020, how='left', left_on='nom', right_on='Région')
merged_T1_T2_2020 = régions.merge(result_2020_T1_T2, how='left', left_on='nom', right_on='Région')
merged_2021_2022 = régions.merge(result_2021_2022, how='left', left_on='nom', right_on='Région')

merged_2019_2020.plot(column='Pourcentage_d\'augmentation_2019_2020', cmap='Reds', linewidth=0.8, ax=axes[0], edgecolor='0.8', legend=True, vmin=vmin, vmax=vmax)
axes[0].set_title("Pourcentage d'augmentation des demandeurs d'emploi par région en 2019/2020",pad=90)
axes[0].set_axis_off()

merged_T1_T2_2020.plot(column='Pourcentage_d\'augmentation_T1_T2_2020', cmap='Reds', linewidth=0.8, ax=axes[1], edgecolor='0.8', legend=True, vmin=vmin, vmax=vmax)
axes[1].set_title("Pourcentage d'augmentation des demandeurs d'emploi par région entre les trimestres 1 et 2 de 2020",pad=90)
axes[1].set_axis_off()

merged_2021_2022.plot(column='Pourcentage_d\'augmentation_2021_2022', cmap='Reds', linewidth=0.8, ax=axes[2], edgecolor='0.8', legend=True, vmin=vmin, vmax=vmax)
axes[2].set_title("Pourcentage d'augmentation des demandeurs d'emploi par région en 2021/2022",pad=90)
axes[2].set_axis_off()

plt.show()

Cette visualisation montre trois cartes de la France, chacune représentant le taux de chômage par région à différentes périodes.

- **La première carte,** à gauche, montre le taux de chômage par région en 2019/2020. Les couleurs vont du blanc au rouge pâle, indiquant des taux de chômage en moyenne faible.

- **La deuxième carte,** au centre, montre le changement du taux de chômage entre les trimestres 1 et 2 de 2020. La légende de couleur varie du blanc au marron, indiquant une diminution du taux de chômage (en blanc et tons clairs) à une augmentation (en tons plus sombres).

- **La troisième carte,** à droite, indique le taux de chômage par région en 2021/2022. Comme la première carte, elle utilise une palette de couleurs allant du blanc au rouge foncé pour représenter les différents taux de chômage.

**En comparant les trois cartes, on peut observer que:**

- La première carte présente une situation de base du taux de chômage avant la pandémie.
- La deuxième carte montre l'impact immédiat de la pandémie de COVID-19 sur l'emploi, avec des augmentations significatives du chômage dans certaines régions.
- La troisième carte semble indiquer une situation de récupération ou de changement continu du taux de chômage, avec certaines régions connaissant une amélioration par rapport à la première carte.

**Les différences notables sont les variations régionales du taux de chômage et la dynamique du changement entre les périodes. Certaines régions semblent avoir été plus affectées que d'autres et la récupération économique pourrait ne pas être uniforme à travers le pays.**

**Conclusion**

En synthèse, l'évolution du nombre de demandeurs d'emploi en France se caractérise par une tendance à la baisse depuis 2021, après une période de hausse notable due à la pandémie de COVID-19, particulièrement au cours du deuxième trimestre de 2020. Cette diminution résulte de divers facteurs, notamment la reprise économique, les mesures de soutien à l'emploi mises en place par le gouvernement français, et l'amélioration de la situation sanitaire. Il est également à noter que la guerre en Ukraine, débutée en 2022, n'a pas eu un impact significatif sur le nombre total de demandeurs d'emploi.

Cependant, il convient de souligner que l'évolution du nombre de demandeurs d'emploi en France présente d'importantes disparités entre les régions. Les régions les plus touchées par le chômage sont celles dotées d'un tissu économique fragile, notamment dans le secteur du tourisme, particulièrement vulnérable pendant la période de la pandémie. Ces disparités soulignent la nécessité d'adopter des approches différenciées pour aborder les défis du chômage, en prenant en compte les spécificités économiques de chaque région.

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">Taux de chômage</h2>

#### Analyse generale

Le taux de chômage constitue un indicateur crucial reflétant la proportion de personnes en âge de travailler qui sont sans emploi et activement à la recherche d'un emploi. En France, ce paramètre est souvent évalué au niveau régional pour capturer les disparités géographiques.

Dans la première section de notre étude, nous entreprendrons une analyse approfondie de l'évolution du taux de chômage à l'échelle nationale, couvrant la période allant de 2017 à 2023. Notre objectif est d'examiner en détail les tendances, les fluctuations et les éventuelles ruptures observées dans le paysage de l'emploi au fil de ces sept années.

Nous allons donc extraire les données sur le taux de chômage qui se trouvent dans la collection `chomage` de la base de données. Ensuite, nous utiliserons ces données pour créer une visualisation permettant une meilleure analyse de ces données.

In [ ]:
chomage = db.chomage.find()
chomage = pd.DataFrame(chomage)
chomage = chomage.drop('_id',axis=1)
chomage_copy = chomage.copy()

In [ ]:
def update_figure(selected_option):
    fig = go.Figure()

    if selected_option == 'Année':
        df_annee = chomage_copy.groupby('Année')['France (hors Mayotte)'].mean().reset_index()
        xlabel = 'Année'
        fig.data = []
        fig.add_trace(go.Scatter(x=df_annee['Année'], y=df_annee['France (hors Mayotte)'],
                                 mode='lines+markers',
                                 name='Taux de Chômage moyen (%)'))
        fig.update_layout(title='Taux de Chômage moyen en France par Année',
                          xaxis=dict(title=xlabel),
                          yaxis=dict(title='Taux de Chômage moyen (%)'))
        
    elif selected_option == 'Trimestre':
        chomage_copy['Année_trimestre'] = chomage_copy['Année'].astype(str) + '-' + chomage_copy['Trimestre_année']
        xlabel = 'Année-Trimestre'
        fig.data = []
        fig = px.bar(chomage_copy, x='Trimestre_année', y='France (hors Mayotte)',
                     labels={'Trimestre_année': xlabel, 'France (hors Mayotte)': 'Taux de Chômage (%)'},
                     title='Taux de Chômage en France par Trimestre',
                     facet_col='Année',
                     category_orders={'Trimestre_année': ['T1', 'T2', 'T3', 'T4']})
    fig.show()

button = widgets.ToggleButtons(
    options=['Année', 'Trimestre'],
    description='Option:',
    disabled=False,
    button_style='primary',
)

interact(update_figure, selected_option=button);

Ce graphique illustre l'évolution du taux de chômage moyen en France par année, de 2017 à 2023, exprimé en pourcentage de la population active. Initialement, une représentation en ligne est utilisée pour afficher les données annuelles, mais grâce à deux boutons situés au-dessus du graphique, il est possible de basculer vers une représentation trimestrielle avec des barres.

**Analyse du graphique sur le taux de chômage par an**

Depuis 2017, le taux de chômage a connu une diminution progressive, passant de 9,4 % à 7,2 % en 2023. Cette diminution peut s'expliquer par les mesures prises par le gouvernement français pour relancer l'économie. Notamment, la pandémie de Covid-19, débutée en 2020 en France, n'a pas eu un impact significatif sur la moyenne annuelle du taux de chômage.

**Analyse du graphique sur le taux de chômage par trimestre**

Une observation attentive révèle une forte augmentation du taux de chômage au troisième trimestre de 2020, en raison de la crise sanitaire liée à la pandémie de Covid-19. De 7,1 % au deuxième trimestre 2020, le taux a grimpé à 9 % au troisième trimestre 2020. Cependant, cette hausse n'est pas perceptible au niveau de l'année entière.

Par la suite, le taux de chômage a amorcé une diminution graduelle pour atteindre 7,4 % au troisième trimestre 2023, expliquée par la reprise de l'activité économique post-crise sanitaire.

**Comparaison du graphique par année et par trimestre**

La comparaison entre les graphiques annuel et trimestriel révèle quelques distinctions notables. Tout d'abord, les variations du taux de chômage sont plus prononcées à court terme qu'à long terme. Par exemple, une augmentation de 0,2 point est observée au troisième trimestre 2023, tandis qu'une diminution de 0,07 point est enregistrée sur l'ensemble de l'année 2023.

En second lieu, le graphique trimestriel permet de mettre en évidence les effets de la saisonnalité sur le taux de chômage. Plus particulièrement, le taux de chômage tend à être plus élevé au premier trimestre qu'au quatrième, attribuable aux contrats saisonniers d'hiver.

Enfin, le graphique trimestriel offre une précision accrue dans le suivi de l'évolution du taux de chômage.

#### Analyse par département

De manière plus approfondie, nous allons représenter graphiquement le taux de chômage enregistré par région pour chaque année en France. Cette opération sera effectuée à l'aide d'un curseur (slider) qui permettra de sélectionner l'année à visualiser.

Notre analyse se penchera sur la représentation graphique de l'évolution du taux de chômage enregistré par région sur une période donnée. À travers un outil interactif de visualisation, tel qu'un curseur (slider), nous permettrons aux utilisateurs de sélectionner aisément l'année qui suscite leur intérêt.

Cette approche graphique facilitera la compréhension des tendances du chômage au fil du temps, tout en mettant en lumière les variations régionales. L'objectif est d'offrir une vision détaillée et nuancée du paysage de l'emploi en France, tout en permettant des comparaisons significatives entre les différentes années.

Nous allons extraire les données sur le taux de chômage par département qui se trouvent dans la collection `taux_chomage_departements` de la base de données.

In [ ]:
taux_chomage_departements = db.taux_chomage_departements.find()
taux_chomage_departements = pd.DataFrame(taux_chomage_departements)
taux_chomage_departements = taux_chomage_departements.drop(columns='_id', axis=1)
taux_chomage_departements.head()

Nous allons allors modifier ces données de facon à avoir tous les valuers en une seule varieble et le nom des departements dans une seule variable pour pouvoir fussionner avec les donnes geographiques plus facileNous allons alors restructurer ces données afin d'avoir toutes les valeurs regroupées dans une seule variable et les noms des départements dans une autre variable. Cela facilitera la fusion avec les données géographiques, permettant ainsi la création d'une carte pour visualiser ces données département par département sur l'ensemble de la France.ment, permettant alors de crer une carte pour visaliser ces donénes par departement sur toute la france.

In [ ]:
taux_chomage_departements = pd.melt(taux_chomage_departements, id_vars=['Année', 'Trimestre'], var_name='Département', value_name='Taux_chomage')
taux_chomage_departements.head()

In [ ]:
taux_chomage_departements['Année'] = taux_chomage_departements['Année'].astype(int)
taux_chomage_departements['Trimestre'] = taux_chomage_departements['Trimestre'].astype(str)
taux_chomage_departements['Taux_chomage'] = pd.to_numeric(taux_chomage_departements['Taux_chomage'], errors='coerce')

In [ ]:
def update_map(year):
    fig, ax = plt.subplots(1, 1, figsize=(10, 8))
    subset = taux_chomage_departements[taux_chomage_departements['Année'] == year]
    merged_df = départements.merge(subset, left_on='nom', right_on='Département', how='left')
    départements.boundary.plot(ax=ax, color='black', linewidth=0.5)
    plot = merged_df.plot(column='Taux_chomage', cmap='Reds', linewidth=0.8, ax=ax, edgecolor='0.8', legend=False)
    ax.set_title(f'Carte du taux de chômage par département en {year}')
    ax.set_axis_off()
    ax.set_aspect('equal')
    vmin, vmax = taux_chomage_departements['Taux_chomage'].min(), taux_chomage_departements['Taux_chomage'].max()
    norm = Normalize(vmin=vmin, vmax=vmax)
    cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])
    cb = ColorbarBase(cax, cmap='Reds', norm=norm, orientation='vertical')
    plt.show()

year_slider = widgets.IntSlider(min=int(taux_chomage_departements['Année'].min()), max=int(taux_chomage_departements['Année'].max()), step=1, value=int(taux_chomage_departements['Année'].min()))

interact(update_map, year=year_slider);

La cartographie du taux de chômage par département en France entre 2017 et 2023 propose une analyse approfondie des disparités régionales. En scrutant la gamme de teintes, qui traduisent les différentes intensités du chômage, une tendance générale se dégage avec des taux généralement plus élevés dans les départements du nord, de l'ouest et du sud du pays.

Parmi les départements les plus touchés au cours de cette période septennale, la Seine-Saint-Denis, le Val-de-Marne et le Pas-de-Calais se distinguent par des taux fluctuant entre 12 et 14 %. Cette persistance souligne l'impératif d'accorder une attention particulière et de mettre en œuvre des mesures spécifiques dans ces zones géographiques.

Notons qu'entre 2017 et 2020, une amélioration du taux de chômage est observée pour la plupart des départements, avec une hausse en 2021 et une nouvelle amélioration par la suite. En 2023, certains départements affichent le taux de chômage le plus bas comparé aux autres années, témoignant d'une dynamique positive dans certaines régions.

Il est crucial de souligner que les variations annuelles, notamment post-COVID en 2021, mettent en lumière la sensibilité du taux de chômage aux évolutions économiques et démographiques. La pandémie a accentué ces dynamiques, incitant à une approche proactive dans l'adaptation des politiques d'emploi.

En approfondissant l'observation, la corrélation entre les taux de chômage élevés et les départements à économie moins développée soulève des questions cruciales. Ces régions sont souvent caractérisées par une main-d'œuvre moins qualifiée et une croissance économique modérée. Ainsi, les politiques de lutte contre le chômage devraient être conçues pour répondre à ces problématiques spécifiques, mettant l'accent sur l'amélioration des systèmes éducatifs, la facilitation de la formation professionnelle et la création d'opportunités d'emploi.

La carte peut être utilisée pour identifier les départements où les politiques de lutte contre le chômage devraient être ciblées. Ces politiques devraient viser à améliorer l'éducation, la formation et les opportunités d'emploi dans ces départements.

**Conclusion**

L'analyse du taux de chômage en France entre 2017 et 2023 dévoile une baisse significative, de 9,4 % à 7,5 %. Cette amélioration, malgré les perturbations de la pandémie en 2020, témoigne de l'efficacité des mesures gouvernementales. La cartographie par département révèle des disparités régionales persistantes, soulignant la nécessité de politiques ciblées. Il est également notable que des fluctuations se manifestent tout au long des trimestres des années, soulignant la dynamique temporelle du marché du travail.

La flexibilité du marché du travail face aux défis, notamment la récupération post-pandémique, souligne sa résilience. La corrélation entre chômage élevé et départements moins développés appelle à des initiatives éducatives et d'emploi spécifiques. En somme, cette analyse offre des insights pour des politiques globales et régionales visant à promouvoir l'emploi et à réduire les inégalités.

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">Comparaison des déclarations préalables à l'embauche par département dans les grands secteurs et des déclarations préalables à l'embauche par an, par sexe et âge</h2>

La différence entre les deux indicateurs est la suivante :

* **Dpae_brut** : il s'agit du nombre total de demandeurs d'emploi inscrits à Pôle emploi, y compris les demandeurs d'emploi en fin de droits, les demandeurs d'emploi en formation, et les demandeurs d'emploi en inactivité partielle.
* **Dpae_cvs** : il s'agit du nombre de demandeurs d'emploi inscrits à Pôle emploi qui sont susceptibles d'être disponibles pour travailler dans les 15 jours.

En d'autres termes, la **Dpae_brut** est un indicateur plus large que la **Dpae_cvs**. Elle inclut des demandeurs d'emploi qui ne sont pas forcément disponibles pour travailler dans un délai court.

La **Dpae_brut** est un indicateur important pour suivre l'évolution de la situation du marché du travail. Elle permet de mesurer l'ampleur du chômage, y compris les demandeurs d'emploi qui ne sont pas immédiatement disponibles pour travailler.

La **Dpae_cvs** est un indicateur plus précis pour mesurer l'offre de travail disponible. Elle permet de cibler les politiques de soutien à l'emploi.

In [ ]:
DPALE_reg = db.DPALE_reg.find()
DPALE_reg = pd.DataFrame(DPALE_reg)
DPALE_reg = DPALE_reg.drop(['_id','nature_de_contrat', 'Duree_de_contrat'], axis=1)
DPALE_reg = DPALE_reg.rename(columns={'region':'Region', 'trimestre':'Trimestre','duree_de_contrat':'Duree_de_contrat','dpae_cvs':'Dpae_cvs'})
DPALE_reg.head()

In [ ]:
somme_par_annee_cvs = DPALE_reg.groupby('annee')['Dpae_cvs'].sum().reset_index()

In [ ]:
DPALE_sexe_age = db.DPALE_sexe_age.find()
DPALE_sexe_age = pd.DataFrame(DPALE_sexe_age)
DPALE_sexe_age = DPALE_sexe_age.drop(['_id','Libelle_de_l_age', 'Libelle_du_sexe'], axis=1)
DPALE_sexe_age.head()

In [ ]:
somme_par_annee_brut = DPALE_sexe_age.groupby('Année')['Dpae_brut'].sum().reset_index()

In [ ]:
gdf = gpd.read_file(url)
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
axs[0].pie(somme_par_annee_cvs['Dpae_cvs'], labels=somme_par_annee_cvs['annee'], autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)
axs[0].legend(title='Année', loc='upper right', bbox_to_anchor=(1.2, 1))
axs[0].set_title('Répartition des DPAE_cvs par année')
axs[1].pie(somme_par_annee_brut['Dpae_brut'], labels=somme_par_annee_brut['Année'], autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)
axs[1].legend(title='Année', loc='upper right', bbox_to_anchor=(1.2, 1))
axs[1].set_title('Répartition des DPAE_brut par année')
plt.tight_layout()
plt.show()

Les deux graphiques présentent la répartition annuelle des demandeurs d'emploi inscrits à Pôle emploi selon deux indicateurs différents : DPAE_brut et DPAE_cvs. Chaque segment correspond à une année spécifique, avec des couleurs distinctes pour chaque année.

**Analyse du graphique DPAE_cvs :**
- L'année 2022 représente la plus grande part avec 16,0%.
- L'année 2018 est la deuxième plus grande avec 15,4%.
- Les années 2017, 2019 et 2023 ont des parts assez proches, respectivement 14,6%, 15,2% et 14%.
- L'année 2020 a la plus petite part avec 11,4%, ce qui pourrait être dû à des facteurs externes comme la pandémie de COVID-19.

**Analyse du graphique DPAE_brut :**
- L'année 2022 occupe la plus grande part avec 18,5%.
- Les années 2018 et 2019 a également une part significative avec respectivement 17,5% et 17.9%.
- Les années 2017,2018 et 2019 sont assez similaires avec respectivement 17,1% 17,6% et 17,9%.
- L'année 2020 a la part la plus faible avec 13,1%.

**Comparaison entre DPAE_brut et DPAE_cvs :**
- Il semble que l'année 2020 ait une part plus faible dans les deux indicateurs, ce qui pourrait indiquer un impact général sur le marché du travail à cette période.
- Les parts de l'année 2018 sont élevées dans les deux graphiques, ce qui suggère un grand nombre de demandeurs d'emploi inscrits cette année-là.
- Les différences entre les parts pour les autres années peuvent refléter la nature des indicateurs. Puisque le DPAE_brut inclut des catégories supplémentaires de demandeurs d'emploi (fin de droits, en formation, inactivité partielle), il est naturel de voir des chiffres globalement plus élevés par rapport au DPAE_cvs qui ne compte que ceux qui sont immédiatement disponibles pour travailler.

<h2 style="padding:15px; color:#FFFF; background: #8a2be2">Partie 2 : L'inflation</h2>

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">L'inflation en général</h2>

L'inflation est une augmentation générale et durable des prix des biens et services dans une économie au cours d'un certain temps (ici nous étudions la période entre 2017 et 2022). En d'autres termes, cela implique une diminution de la capacité d'achat de la monnaie, ce qui entraîne une augmentation des prix des biens et services courants. Les indices des prix à la consommation, qui reflètent les variations moyennes des prix d'un panier représentatif de biens et services que les consommateurs achètent fréquemment, sont généralement utilisés pour mesurer l'inflation.

L'inflation peut être causée par divers facteurs, tels que la demande excédentaire, qui est définie comme une demande totale supérieure à l'offre disponible, les coûts de production croissants, la hausse des salaires, la dépréciation de la monnaie, les pressions sur les matières premières, etc. Dans une économie, un niveau d'inflation modéré est souvent considéré comme normal car il stimule la consommation et l'investissement. Cependant, une inflation excessive peut avoir des conséquences négatives telles que la perte de pouvoir d'achat, l'incertitude économique et des distorsions dans les décisions d'investissement.

Dans une première partie on récupère les données de la collection MongoDB "inflation" et on stocke le résultat dans une variable appelée "inflation". on convertit les données MongoDB (qui sont sous forme de dictionnaires) en un DataFrame pandas. On convertit toutes les colonnes du DataFrame en types numériques. Les valeurs qui ne peuvent pas être converties sont remplacées par NaN (Not a Number). On supprime les colonnes qui contiennent au moins une valeur NaN. Cela élimine les colonnes qui ne peuvent pas être converties en nombres. Finalement, on affiche le DataFrame "inflation" après les transformations. 

In [ ]:
inflation = db.inflation.find()
inflation = pd.DataFrame(inflation)
inflation = inflation.apply(pd.to_numeric, errors='coerce', downcast='integer')
inflation = inflation.dropna(axis=1)
inflation

In [ ]:
plt.plot(inflation['Année'], inflation["taux_d'inflation"], marker='o')

plt.axvline(x=2020, color='r', linestyle='--', label='Pandémie Covid')
plt.axvline(x=2022, color='b', linestyle='--', label='Début Guerre Ukraine')

plt.title("Evolution de l'inflation entre 2017 et 2022")
plt.xlabel('Années')
plt.ylabel('Inflation')
plt.legend()
plt.show()

**Les principales conclusions de l'analyse du graphique sont les suivantes :**

Alors que les fluctuations de l'indice des prix à la consommation, la principale mesure de l'inflation, étaient relativement faibles d'une année sur l'autre jusqu'en 2021, les prix à la consommation augmenteront considérablement en moyenne annuelle en 2022. Après une croissance de +1,6 % en 2021 et de +0,5 % en 2020, l'inflation annuelle s'élève à 5,2%.

L'inflation n'a pas dépassé le seuil de 2 % de 2017 à 2021, avec des causes extérieures en général bien identifiées, telles que les variations des conditions climatiques, l'environnement géopolitique ou les décisions de santé publique (tabac). 

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">Prédiction de l'inflation entre 2023 et 2028</h2>

Pour la partie sur les prédictions, dans une première partie on récupère les données de la collection MongoDB "inflation_prediction" et on stocke le résultat dans une variable appelée "inflation_pred". On convertit les données MongoDB (qui sont sous forme de dictionnaires) en un DataFrame pandas.  On convertit toutes les colonnes du DataFrame en types numériques. Les valeurs qui ne peuvent pas être converties sont remplacées par NaN (Not a Number). On supprime les colonnes qui contiennent au moins une valeur NaN. Cela élimine les colonnes qui ne peuvent pas être converties en nombres.

In [ ]:
inflation_pred = db.inflation_prediction.find()
inflation_pred = pd.DataFrame(inflation_pred)
inflation_pred = inflation_pred.apply(pd.to_numeric, errors='coerce', downcast='integer')
inflation_pred = inflation_pred.dropna(axis=1)
inflation_pred

On concaténate les DataFrames "inflation" et "inflation_pred".

In [ ]:
inflation_total = pd.concat([inflation,inflation_pred], ignore_index=True)
inflation_total

In [ ]:
plt.bar(inflation_total['Année'], inflation_total['taux_d\'inflation'])
plt.xlabel('Année')
plt.ylabel('Taux d\'inflation')
plt.title('Taux d\'inflation par année')
plt.axvline(x=2020, color='r', linestyle='--', label='Pandémie Covid')
plt.axvline(x=2022, color='b', linestyle='--', label='Début Guerre Ukraine')
plt.legend()
plt.show()

Les prédictions de l'inflation sont basées sur une variété de facteurs, y compris :

- **Les données historiques sur l'inflation.** Les prédicteurs examinent généralement les tendances historiques de l'inflation pour tenter de prédire ce qu'elle sera à l'avenir.

- **Les facteurs économiques actuels.** Les prédicteurs examinent également les facteurs économiques actuels qui pourraient affecter l'inflation, tels que la croissance économique, le taux de chômage et les taux d'intérêt.

- **Les facteurs économiques actuels.** Les prédicteurs examinent également les facteurs économiques actuels qui pourraient affecter l'inflation, tels que la croissance économique, le taux de chômage et les taux d'intérêt.

Dans le cas du conflit israélo-palestinien, il est possible que les prédictions de l'inflation soient biaisées en raison de l'incertitude du conflit. Si le conflit s'intensifie, il pourrait entraîner une hausse des prix des matières premières et des biens énergétiques, ce qui pourrait entraîner une hausse de l'inflation. Cependant, il est également possible que le conflit se calme, ce qui pourrait entraîner une baisse de l'inflation.

Il est important de noter que les prédictions de l'inflation sont toujours des estimations et qu'elles peuvent être inexactes. Les prédicteurs utilisent leurs meilleures estimations des facteurs qui affectent l'inflation, mais ils ne peuvent jamais être sûrs de ce qui se passera à l'avenir.

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">L'évolution de la consommation mensuelle des ménages en biens - Alimentaire hors tabac et de la consommation mensuelle des ménages en biens et Énergie, eau, déchets, cokéfaction et raffinage</h2>

Autout debut, on récupère les données de la collection MongoDB "alimentation_consommation" et on stocke le résultat dans une variable appelée "conso_alimentaire". On convertit les données MongoDB (qui sont sous forme de dictionnaires) en un DataFrame pandas. On supprime la colonne "_id" du DataFrame "conso_alimentaire". On utilise la colonne 'Année-Mois' pour extraire l'année et la stocker dans une nouvelle colonne appelée 'Année'. On supprime la colonne 'Année-Mois' du DataFrame "conso_alimentaire". On convertit la colonne 'Consommation_Alimentaire' en un type numérique. Les valeurs qui ne peuvent pas être converties sont remplacées par NaN (Not a Number). 

In [ ]:
conso_alimentaire = db.alimentation_consommation.find()
conso_alimentaire = pd.DataFrame(conso_alimentaire)
conso_alimentaire.drop('_id', axis=1, inplace=True)
conso_alimentaire['Année'] = pd.to_datetime(conso_alimentaire['Année-Mois']).dt.year
conso_alimentaire.drop('Année-Mois', axis=1, inplace=True)
conso_alimentaire['Consommation_Alimentaire'] = pd.to_numeric(conso_alimentaire['Consommation_Alimentaire'], errors='coerce')
conso_alimentaire.head()

In [ ]:
plt.figure(figsize=(10, 7))
grouped_data = conso_alimentaire.groupby('Année')['Consommation_Alimentaire'].mean().reset_index()
plt.boxplot([conso_alimentaire[conso_alimentaire['Année'] == year]['Consommation_Alimentaire'] for year in grouped_data['Année']],
            positions=grouped_data['Année'], showmeans=True, meanline=True, meanprops={"marker":"o", "markerfacecolor":"white", "markeredgecolor":"black"})
plt.axvline(x=2020, color='r', linestyle='--', label='Pandémie Covid')
plt.axvline(x=2022, color='b', linestyle='--', label='Début Guerre Ukraine')
plt.title('Boîte à moustaches de la Consommation Alimentaire moyenne par année')
plt.xlabel('Année')
plt.ylabel('Consommation Alimentaire')
plt.legend()
plt.show()

**Les principales conclusions de l'analyse du graphique sont les suivantes :**

- La consommation alimentaire a connu une grosse baisse en 2022, passant de 16,2 kg par habitant en moyenne à 15,7 kg par habitant en moyenne.
- Celle-ci a encore plus baissée en 2023 et à atteint environ 15 kg par habitant en moyenne. Si on compare entre 2020 et 2023, on a perdu plus d'un kilogramme.
- Cette baisse est due à une combinaison de facteurs, notamment la hausse des prix des produits alimentaires, qui ont réduit le pouvoir d'achat des ménages, et les perturbations de la chaîne d'approvisionnement causées par la pandémie de COVID-19 et la guerre en Ukraine.

**Pandémie de COVID-19**

La pandémie de COVID-19 a eu un impact important sur l'économie mondiale, y compris sur la consommation alimentaire. Les mesures de confinement et de distanciation sociale ont perturbé les habitudes de consommation des ménages et ont entraîné une baisse de la consommation de certains biens et services, notamment de la restauration et des loisirs.

**Guerre en Ukraine**

La guerre en Ukraine a également contribué à la baisse de la consommation alimentaire. La guerre a entraîné une hausse des prix des produits alimentaires, notamment des céréales, qui sont une source importante de calories pour les ménages à faible revenu.

**Conclusion**

Le graphique montre que la consommation alimentaire en France a connu une baisse à partit de 2021. Cette baisse est due à une combinaison de facteurs, notamment la hausse des prix des produits alimentaires et les perturbations de la chaîne d'approvisionnement.

Ensuite, on récupère les données de la collection MongoDB "energie_consommation" et on stocke le résultat dans une variable appelée "conso_energie". On convertit les données MongoDB (qui sont sous forme de dictionnaires) en un DataFrame pandas. Supprime la colonne "_id" du DataFrame "conso_energie". Utilise la colonne 'Année-Mois' pour extraire l'année et la stocker dans une nouvelle colonne appelée 'Année'. On supprime la colonne 'Année-Mois' du DataFrame "conso_energie". On convertit la colonne 'Consommation_Energie' en un type numérique. Les valeurs qui ne peuvent pas être converties sont remplacées par NaN (Not a Number). 

In [ ]:
conso_energie = db.energie_consommation.find()
conso_energie = pd.DataFrame(conso_energie)
conso_energie.drop('_id', axis=1, inplace=True)
conso_energie['Année'] = pd.to_datetime(conso_energie['Année-Mois']).dt.year
conso_energie.drop('Année-Mois', axis=1, inplace=True)
conso_energie['Consommation_Energie'] = pd.to_numeric(conso_energie['Consommation_Energie'], errors='coerce')
conso_energie.head()

In [ ]:
plt.figure(figsize=(10, 7))
grouped_data = conso_energie.groupby('Année')['Consommation_Energie'].mean().reset_index()
plt.boxplot([conso_energie[conso_energie['Année'] == year]['Consommation_Energie'] for year in grouped_data['Année']],
            positions=grouped_data['Année'], showmeans=True, meanline=True, meanprops={"marker":"o", "markerfacecolor":"white", "markeredgecolor":"black"})
plt.axvline(x=2020, color='r', linestyle='--', label='Pandémie Covid')
plt.axvline(x=2022, color='b', linestyle='--', label='Début Guerre Ukraine')
plt.title("Boîte à moustaches de la Consommation Energétique moyenne par année")
plt.xlabel('Année')
plt.ylabel('Consommation Energie')
plt.legend()
plt.show()

**Les principales conclusions de l'analyse du graphique sont les suivantes :**

- La consommation d'énergie moyenne a connu une baisse significative en 2022, passant d'environ 8,4 kWh par habitant en moyenne à environ 8 kWh par habitant en moyenne.
- Cette baisse est due à une combinaison de facteurs, notamment la baisse de la disponibilité du parc nucléaire, la hausse des prix de l'énergie et les mesures de sobriété énergétique prises par le gouvernement.

**Pandémie de COVID-19**

La pandémie de COVID-19 a eu un impact important sur l'économie mondiale, y compris sur la consommation d'énergie. Les mesures de confinement et de distanciation sociale ont perturbé les habitudes de consommation des ménages et ont entraîné une baisse de la consommation de certains biens et services, notamment de la restauration et des loisirs.

**Guerre en Ukraine**

La guerre en Ukraine a également contribué à la baisse de la consommation d'énergie. La guerre a entraîné une hausse des prix des produits énergétiques, notamment du pétrole et du gaz, ce qui a incité les ménages à réduire leur consommation d'énergie.

**Mesures de sobriété énergétique**

Le gouvernement français a également pris des mesures pour réduire la consommation d'énergie, notamment en encourageant les ménages à mieux isoler leurs maisons et à utiliser des appareils électroménagers plus efficaces.

**Conclusion**

Le graphique montre que la consommation d'énergie moyenne a connu une baisse significative en 2022. Cette baisse est due à une combinaison de facteurs, notamment la baisse de la disponibilité du parc nucléaire, la hausse des prix de l'énergie et les mesures de sobriété énergétique prises par le gouvernement.

**Remarques**

- Le graphique ne montre pas la consommation d'énergie par source d'énergie. Il serait intéressant de voir comment la consommation d'électricité, de gaz naturel et de carburants a évolué au fil du temps.
- Le graphique ne montre pas la consommation d'énergie par secteur. Il serait intéressant de voir comment la consommation d'énergie dans les secteurs résidentiel, industriel et tertiaire a évolué au fil du temps.

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">Comparaison de la consommation alimentaire et énergétique</h2>

Afin de faciliter la comparaison de nos données, nous devons les normaliser en les ramenant à une échelle commune. La normalisation implique la réduction et la standardisation des données, ce qui permet de les rendre comparables en éliminant les différences d'échelle. Cette étape nous offre la possibilité de comparer les tendances et les variations dans nos différentes séries de données de manière plus équitable.

On calcule la moyenne de la consommation alimentaire par année. On calcule la moyenne de la consommation d'énergie par année. On calcule la moyenne globale de la consommation alimentaire et de la consommation d'énergie. On calcule l'écart type global de la consommation alimentaire et de la consommation d'énergie. On calcule la valeur moyenne de la consommation alimentaire standardisée. On calcule la valeur moyenne de la consommation d'énergie standardisée. 

In [ ]:
conso_ali_moyenne = conso_alimentaire.groupby('Année')['Consommation_Alimentaire'].mean().reset_index()
conso_ene_moyenne = conso_energie.groupby('Année')['Consommation_Energie'].mean().reset_index()
mean_value_ali,mean_value_ene = conso_ali_moyenne['Consommation_Alimentaire'].mean(),conso_ene_moyenne['Consommation_Energie'].mean()
std_value_ali,std_value_ene = conso_ali_moyenne['Consommation_Alimentaire'].std(),conso_ene_moyenne['Consommation_Energie'].std()
conso_ali_moyenne['valeur_moyenne_ali_standardisee'] = (conso_ali_moyenne['Consommation_Alimentaire'] - mean_value_ali) / std_value_ali
conso_ene_moyenne['valeur_moyenne_ene_standardisee'] = (conso_ene_moyenne['Consommation_Energie'] - mean_value_ene) / std_value_ene

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(conso_ali_moyenne['Année'], conso_ali_moyenne["valeur_moyenne_ali_standardisee"], marker='o', color="g", label='Consommation Alimentaire (standardisée)')
plt.plot(conso_ene_moyenne['Année'], conso_ene_moyenne["valeur_moyenne_ene_standardisee"], marker='o', color="purple", label='Consommation Energétique (standardisée)')

plt.axvline(x=2020, color='r', linestyle='--', label='Pandémie Covid')
plt.axvline(x=2022, color='b', linestyle='--', label='Début Guerre Ukraine')

plt.title("Evolution de la consommation alimentaire et énergétique (standardisées) entre 2017 et 2022")
plt.xlabel('Années')
plt.ylabel('Valeurs standardisée')
plt.legend()
plt.show()

**Les principales conclusions de l'analyse du graphique sont les suivantes :**

- La consommation alimentaire a connu une légère baisse en 2022, tandis que la consommation énergétique a fortement diminué.
- Cette différence s'explique en partie par la hausse des prix des produits alimentaires, qui ont réduit le pouvoir d'achat des ménages.
- La hausse des prix des produits énergétiques, notamment du pétrole et du gaz, a également contribué à la baisse de la consommation énergétique.

**Pandémie de COVID-19**

La pandémie de COVID-19 a eu un impact important sur l'économie mondiale, y compris sur la consommation. Les mesures de confinement et de distanciation sociale ont perturbé les habitudes de consommation des ménages et ont entraîné une baisse de la consommation de certains biens et services, notamment de la restauration et des loisirs.

**Guerre en Ukraine**

La guerre en Ukraine a également contribué à la baisse de la consommation alimentaire. La guerre a entraîné une hausse des prix des produits alimentaires, notamment des céréales, qui sont une source importante de calories pour les ménages à faible revenu.

**Perspectives de la consommation (nous verons ça dans la dernière partie)**

La Banque de France s'attend à ce que la consommation alimentaire en France se stabilise en 2023, tandis que la consommation énergétique devrait continuer à augmenter. Cependant, il est possible que la consommation reste affectée par la hausse des prix des produits alimentaires et énergétiques.

**Conclusion**

Le graphique montre que la consommation alimentaire et énergétique en France a évolué de manière différente entre 2017 et 2022. La consommation alimentaire a connu une légère baisse, tandis que la consommation énergétique une très grosse notamment en 2020 alors que la consommation alimentaire augmentait. Cette différence s'explique en partie par la hausse des prix des produits alimentaires, qui ont réduit le pouvoir d'achat des ménages. La hausse des prix des produits énergétiques, notamment du pétrole et du gaz, a également contribué à la baisse de la consommation énergétique. Ainsi entre 2019 et 2021 les tendances sont inversées, quand la consommation alimentaire augmente, la consommation énergétique diminue et inversement avant que toutes les deux diminuent à partir de 2021.

<h2 style="padding:15px; color:#FFFF; background: #bf7de8">Comparaison de l'inflation, les Indices des prix à la consommation et le cout du travail</h2>

Un indicateur économique appelé Indice des prix à la consommation mesure les variations moyennes des prix des biens et services consommés par les ménages au fil du temps. Il est employé pour évaluer l'inflation, qui est l'augmentation générale des prix. En général, l'IPC est calculé en comparant les prix d'un panier représentatif de biens et de services à différentes périodes.

Correction des valeurs saisonnières : La correction des valeurs saisonnières est une technique statistique utilisée pour supprimer les variations saisonnières régulières et prévisibles des données temporelles. En ajustant les données pour tenir compte des effets saisonniers tels que les fluctuations liées aux saisons de l'année (par exemple, les vacances, les saisons agricoles, etc.), elle vise à isoler les tendances sous-jacentes et les variations cycliques. Cela permet une analyse plus efficace des tendances et des variations cycliques.

L'inflation sous-jacente, également appelée inflation de base, est une mesure de l'inflation qui ne prend pas en compte certains facteurs volatils des indices des prix à la consommation, comme les prix de l'énergie et de l'alimentation. L'inflation sous-jacente vise à fournir une mesure plus stable et à long terme de l'évolution des prix en éliminant ces éléments jugés plus susceptibles de subir des fluctuations temporaires, permettant une meilleure évaluation de la pression inflationniste fondamentale dans une économie.

Afin de faciliter la comparaison de nos données, nous devons les normaliser en les ramenant à une échelle commune. La normalisation implique la réduction et la standardisation des données, ce qui permet de les rendre comparables en éliminant les différences d'échelle. Cette étape nous offre la possibilité de comparer les tendances et les variations dans nos différentes séries de données de manière plus équitable.

Autout debut, on récupère les données de la collection MongoDB "EFIPC" et on stocke le résultat dans une variable appelée "EFIPC". On convertit les données MongoDB (qui sont sous forme de dictionnaires) en un DataFrame pandas. On liste des colonnes à supprimer, dans ce cas, uniquement "_id". On supprime la colonne "_id" du DataFrame "EFIPC". On convertit la colonne 'periode' en un objet de type date. On utilise la colonne 'periode' pour extraire l'année et la stocker dans une nouvelle colonne appelée 'année'. On supprime la colonne 'periode' du DataFrame "EFIPC". On calcule la moyenne de la colonne 'valeur' par année et réinitialise l'index. On renomme les colonnes pour plus de clarté. On calcule la moyenne globale de la colonne 'valeur_moyenne'. On calcule l'écart type global de la colonne 'valeur_moyenne'. On calcule la valeur moyenne standardisée en soustrayant la moyenne globale et en divisant par l'écart type global. On affiche le DataFrame "EFIPC_moyennes" après les transformations. 

In [ ]:
EFIPC = db.EFIPC.find()
EFIPC = pd.DataFrame(EFIPC)
columns_to_drop = ['_id']
EFIPC = EFIPC.drop(columns=columns_to_drop, axis=1)
EFIPC['periode'] = pd.to_datetime(EFIPC['periode'])
EFIPC['année'] = EFIPC['periode'].dt.year
EFIPC.drop('periode', axis=1, inplace=True)
EFIPC_moyennes = EFIPC.groupby('année')['valeur'].mean().reset_index()
EFIPC_moyennes.columns = ['année', 'valeur_moyenne']
mean_value = EFIPC_moyennes['valeur_moyenne'].mean()
std_value = EFIPC_moyennes['valeur_moyenne'].std()
EFIPC_moyennes['valeur_moyenne_standardisee'] = (EFIPC_moyennes['valeur_moyenne'] - mean_value) / std_value
EFIPC_moyennes

On calcule la moyenne de la colonne "taux_d'inflation". On calcule l'écart type de la colonne "taux_d'inflation". On calcule la valeur standardisée en soustrayant la moyenne de la colonne "taux_d'inflation" et en divisant par son écart type. 


In [ ]:
mean_value = inflation["taux_d'inflation"].mean()
std_value = inflation["taux_d'inflation"].std()
inflation['taux_standardise'] = (inflation["taux_d'inflation"] - mean_value) / std_value
inflation

L'indice du coût du travail (ICT) est un indicateur économique qui mesure les variations du coût total de la main-d'œuvre nécessaire pour produire un bien ou un service donné. Cet indice est souvent exprimé en pourcentage et permet de suivre les évolutions du coût du travail au fil du temps.

Plus spécifiquement, l'indice du coût du travail - coût horaire - ensemble des secteurs représente le coût horaire moyen de la main-d'œuvre dans tous les secteurs de l'économie. Il englobe les coûts salariaux directs (salaires bruts) ainsi que d'autres éléments tels que les cotisations sociales patronales, les avantages sociaux et les taxes liées à l'emploi. Cet indice est utilisé pour évaluer la pression exercée sur les coûts de production liés à la main-d'œuvre.

L'évolution de l'ICT - coût horaire - ensemble des secteurs peut être un indicateur important pour les entreprises, les économistes et les responsables politiques, car elle peut influencer les décisions liées à l'investissement, à l'emploi et aux politiques salariales. Un ICT en hausse peut indiquer une pression accrue sur les coûts pour les entreprises, tandis qu'un ICT en baisse peut suggérer une relative stabilité ou une diminution des coûts du travail.

On récupère les données de la collection MongoDB "indice_cout" et on stocke le résultat dans une variable appelée "indice_cout_travail". On convertit les données MongoDB (qui sont sous forme de dictionnaires) en un DataFrame pandas. On liste des colonnes à supprimer, dans ce cas, uniquement "_id". Supprime la colonne "_id" du DataFrame "indice_cout_travail". On convertit la colonne "Indice_cout_travail" en valeurs numériques en gérant les erreurs (convertit en NaN si la conversion échoue). On calcule la moyenne de la colonne "Indice_cout_travail". On calcule l'écart type de la colonne "Indice_cout_travail". On calcule la valeur standardisée en soustrayant la moyenne de la colonne "Indice_cout_travail" et en divisant par son écart type. Affiche les premières lignes du DataFrame "indice_cout_travail" après les transformations. 


In [ ]:
indice_cout_travail = db.indice_cout.find()
indice_cout_travail = pd.DataFrame(indice_cout_travail)
columns_to_drop = ['_id']
indice_cout_travail = indice_cout_travail.drop(columns=columns_to_drop, axis=1)
indice_cout_travail["Indice_cout_travail"] = pd.to_numeric(indice_cout_travail["Indice_cout_travail"], errors='coerce', downcast='integer')
mean_value = indice_cout_travail["Indice_cout_travail"].mean()
std_value = indice_cout_travail["Indice_cout_travail"].std()
indice_cout_travail['Taux_standardise'] = (indice_cout_travail["Indice_cout_travail"] - mean_value) / std_value
indice_cout_travail.head()

On utilise la méthode str.split pour diviser les valeurs de la colonne "Année" en fonction du caractère '-', puis on utilise expand=True pour créer de nouvelles colonnes à partir des parties séparées. Cela extrait l'année. On convertit la colonne "Taux_standardise" en valeurs numériques, en gérant les erreurs (convertit en NaN si la conversion échoue). On convertit la colonne "Année" en un objet de type date, en utilisant le format '%Y' pour indiquer que les valeurs représentent des années. On utilise la méthode dt.year pour extraire l'année uniquement. On calcule la moyenne de la colonne "Taux_standardise" par année et réinitialise l'index. On affiche les premières lignes du DataFrame "indice_cout" après les transformations. 

In [ ]:
indice_cout_travail[['Année']] = indice_cout_travail['Année'].str.split('-', expand=True)
indice_cout_travail["Taux_standardise"] = pd.to_numeric(indice_cout_travail["Taux_standardise"], errors='coerce')
indice_cout_travail["Année"] = pd.to_datetime(indice_cout_travail["Année"], format='%Y', errors='coerce')
indice_cout_travail["Année"] = indice_cout_travail["Année"].dt.year
indice_cout = indice_cout_travail.groupby('Année')['Taux_standardise'].mean().reset_index()
indice_cout.head()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(inflation['Année'], inflation["taux_standardise"], marker='o', color="g", label='Inflation (standardisée)')
plt.plot(EFIPC_moyennes['année'], EFIPC_moyennes["valeur_moyenne_standardisee"], marker='o', color="purple", label='Consommation (standardisée)')
plt.plot(indice_cout["Année"], indice_cout["Taux_standardise"], marker='o', label='Cout travail (standardisée)')

plt.axvline(x=2020, color='r', linestyle='--', label='Pandémie Covid')
plt.axvline(x=2022, color='b', linestyle='--', label='Début Guerre Ukraine')

plt.title("Evolution de l'inflation, de consommation et du coût du travail (standardisées) entre 2017 et 2023")
plt.xlabel('Années')
plt.ylabel('Valeurs standardisée')
plt.legend()
plt.show()

**Les principales conclusions de l'analyse du graphique sont les suivantes :**

- Le graphique montre que l'inflation a augmenté de manière significative entre 2017 et 2023, passant de -0,5 % en 2017 à 2 % en 2022. Cette augmentation de l'inflation a été principalement causée par la pandémie de COVID-19 et la guerre en Ukraine.

- La consommation a également augmenté entre 2017 et 2023, mais de manière moins significative que l'inflation. La consommation a augmenté de -1 % en 2017 à 1.7 % en 2023. Cette augmentation de la consommation a été principalement causée par la reprise économique après la pandémie de COVID-19.

- Le coût du travail a également augmenté entre 2017 et 2023, mais de manière moins significative que l'inflation. Le coût du travail a augmenté de -1,4 % en 2017 à 1,7 % en 2022. Cette augmentation du coût du travail a été principalement causée par la hausse des salaires et des charges sociales.

- Le graphique montre donc que l'inflation a augmenté de manière significative entre 2017 et 2023, tandis que la consommation et le coût du travail ont augmenté de manière moins significative. Cette évolution est due à la pandémie de COVID-19 et à la guerre en Ukraine.

**Voici quelques interprétations supplémentaires du graphique :**

- L'augmentation de l'inflation a réduit le pouvoir d'achat des ménages.
- L'augmentation de l'inflation a également entraîné une augmentation des coûts de production pour les entreprises.
- L'augmentation de la consommation montre que les ménages ont continué à dépenser de l'argent, malgré l'augmentation de l'inflation.
- L'augmentation du coût du travail montre que les entreprises ont continué à investir dans leurs salariés, malgré l'augmentation de l'inflati

**Pandémie de COVID-19.**

La pandémie de COVID-19 a eu un impact important sur l'économie mondiale, y compris sur l'inflation. Les mesures de confinement et de distanciation sociale ont perturbé les chaînes d'approvisionnement et ont entraîné une hausse des prix des biens et services.

**Guerre en Ukraine**

La guerre en Ukraine a également contribué à la hausse de l'inflation. La guerre a entraîné une hausse des prix des produits énergétiques, notamment du pétrole et du gaz.

**Perturbations de la chaîne d'approvisionnement**

Les perturbations de la chaîne d'approvisionnement ont également contribué à la hausse de l'inflation. Les perturbations ont été causées par la pandémie de COVID-19, la guerre en Ukraine et d'autres facteurs.

**Perspectives de l'inflation (nous l'analyserons dans la quatrième partie)**

La Banque centrale européenne (BCE) s'attend à ce que l'inflation en France se stabilise à un niveau légèrement inférieur à 2 % à moyen terme. Cependant, il est possible que l'inflation reste élevée pendant encore quelques mois.

**Remarque**

On a remarqué précedemment que la consommation alimentaire et énergétique avaient diminué entre 2017 et 2023 cependant, dans notre dernier graphique, on remarque que la consommation générale a elle augmenté.

Il y a plusieurs facteurs qui pourraient expliquer pourquoi la consommation générale augmente, mais que la consommation alimentaire et énergétique a baissé entre 2017 et 2023. Voici quelques explications possibles :

1. **Changements dans les habitudes de consommation :** Les tendances en matière de consommation peuvent évoluer au fil du temps en raison de changements dans les préférences des consommateurs. Il se peut que, malgré une augmentation globale de la consommation, les gens aient modifié leurs habitudes alimentaires et énergétiques, optant peut-être pour des choix plus durables ou axés sur la santé.

2. **Sensibilisation environnementale :** Une prise de conscience croissante des questions environnementales peut entraîner une réduction de la consommation énergétique. Les consommateurs pourraient être plus attentifs à la réduction de leur empreinte carbone et faire des choix énergétiques plus économes.

3. **Innovations technologiques :** Des avancées technologiques peuvent entraîner une amélioration de l'efficacité énergétique des appareils et des processus de production, ce qui pourrait contribuer à une baisse de la consommation énergétique.

4. **Changements économiques :** Les conditions économiques, telles que les fluctuations des prix des denrées alimentaires et de l'énergie, peuvent influencer les choix de consommation. Une période de ralentissement économique pourrait conduire à des ajustements dans les dépenses, même si la consommation globale augmente.

5. **Préoccupations de santé :** Des préoccupations croissantes concernant la santé pourraient influencer les choix alimentaires, conduisant à une baisse de la consommation alimentaire liée à des produits spécifiques.

6. **Politiques et réglementations :** Des politiques et réglementations gouvernementales, telles que des incitations à l'efficacité énergétique ou des campagnes de sensibilisation à la nutrition, pourraient influencer les comportements de consommation.

<h2 style="padding:15px; color:#FFFF; background: #8a2be2">Partie 3 : Les modèles</h2>

Nous souhaitons créer un modèle permettant de prédire le taux d'inflation.

Dans une première partie on liste des noms des collections. Ensuite on fusionnne des DataFrames "evolution_salaire_homme" et "evolution_salaire_femme" sur la colonne 'Année', en ajoutant des suffixes aux colonnes pour différencier les valeurs correspondantes pour les hommes et les femmes. On fusionne aussi le DataFrame résultant avec "evolution_salaire" sur la colonne 'Année'. Ensuite, on fusionne le DataFrame résultant avec "indice_cout" sur la colonne 'Année'. Et finalement, on fusionne le DataFrame résultant avec "inflation" sur la colonne 'Année'. Pour finir, on supprime les colonnes qui contiennent des valeurs NaN. 


In [ ]:
inflation = db.inflation.find()
inflation = pd.DataFrame(inflation)
inflation = inflation.apply(pd.to_numeric, errors='coerce', downcast='integer')

In [ ]:
evolution_salaire_homme = db.evolution_salaire_homme.find()
evolution_salaire_homme = pd.DataFrame(evolution_salaire_homme)
evolution_salaire_homme = evolution_salaire_homme.apply(pd.to_numeric, errors='coerce', downcast='integer')
evolution_salaire_homme = evolution_salaire_homme.drop('_id',axis=1)

In [ ]:
evolution_salaire_femme = db.evolution_salaire_femme.find()
evolution_salaire_femme = pd.DataFrame(evolution_salaire_femme)
evolution_salaire_femme = evolution_salaire_femme.apply(pd.to_numeric, errors='coerce', downcast='integer')
evolution_salaire_femme = evolution_salaire_femme.drop('_id',axis=1)

In [ ]:
evolution_salaire = db.evolution_salaire.find()
evolution_salaire = pd.DataFrame(evolution_salaire)
evolution_salaire = evolution_salaire.apply(pd.to_numeric, errors='coerce', downcast='integer')
evolution_salaire = evolution_salaire.drop('_id',axis=1)

In [ ]:
indice_cout = db.indice_cout.find()
indice_cout = pd.DataFrame(indice_cout)
indice_cout = indice_cout.drop(['_id'],axis=1)
indice_cout['Indice_cout_travail'] = pd.to_numeric(indice_cout['Indice_cout_travail'], errors='coerce')
indice_cout['Année'] = pd.to_datetime(indice_cout['Année'], format='%Y').dt.year
indice_cout = indice_cout.groupby('Année')['Indice_cout_travail'].mean().reset_index()

In [ ]:
collection_names = ['evolution_salaire', 'evolution_salaire_femme', 'evolution_salaire_homme', 'indice_cout', 'inflation', 'inflation_prediction']
modele1 = pd.merge(evolution_salaire_homme, evolution_salaire_femme, on='Année', suffixes=('_homme', '_femme'))
modele1 = pd.merge(modele1,evolution_salaire, on='Année')
modele1=pd.merge(modele1,indice_cout, on='Année')
modele1=pd.merge(modele1,inflation, on='Année')
modele1 = modele1.dropna(axis=1)
modele1

In [ ]:
X = modele1[['Evolution_salaire_homme', 'Evolution_salaire_femme', 'Evolution_salaire', 'Indice_cout_travail']]
X = sm.add_constant(X)
y = modele1["taux_d'inflation"]
model = sm.OLS(y, X).fit()
model.summary()

### Analyse des Résultats de Régression OLS (Taux d'Inflation)

#### Modèle de Régression OLS - Taux d'Inflation

Les résultats de la régression linéaire ordinaire (OLS) pour le modèle de taux d'inflation sont les suivants :

- **R-squared (R²) :** 0.948, indiquant un ajustement fort du modèle aux données.

- **Coefficients :**
  - Constante :15.7350
  - Évolution_salaire_homme : 19.2960
  - Évolution_salaire_femme : 14.5726
  - Évolution_salaire : -33.3270
  - *Indice_cout_travail :* -0.1816

- **Valeurs p (P>|t|) :**
  - Constante : 0.774
  - Évolution_salaire_homme  0.568
  - Évolution_salaire_femme : 0.510
  - Évolution_salaire : 0.548
  - Indice_cout_travail : 0.762

- **Statistique F :** 4.562 (Prob (F-statistic): 0.336)

- **Commentaires :**
  - Le R² élevé suggère que le modèle explique bien la variance dans le taux d'inflation.
  - Aucun des coefficients n'est significatif à un niveau de confiance de 95% (p-value > 0.05).
  - La statistique F relativement basse peut suggérer que le modèle ne contribue pas significativement à l'explication de la variance.

- **Tests Statistiques Supplémentaires :**
  - Test Omnibus : La valeur élevée indique que les résidus peuvent suivre une distribution normale.
  - Test Durbin-Watson : La valeur de 2.096 suggère une faible autocorrélation des résidus.
  - Test Jarque-Bera (JB) : La p-value élevée indique que les résidus peuvent suivre une distribution normale.

#### Considérations Générales :

- Le R² élevé montre que le modèle a une bonne capacité à expliquer la variation dans le taux d'inflation.
- Aucun des coefficients n'est significatif individuellement, ce qui peut indiquer que l'ajout de ces variables ne contribue pas de manière significative à la prédiction du taux d'inflation.
- La faible valeur de la statistique F suggère que le modèle dans son ensemble n'est pas statistiquement significatif.

**Note :** Il est recommandé de recueillir davantage de données pour renforcer la robustesse des résultats de la régression.

In [ ]:
X = modele1[['Evolution_salaire_homme', 'Evolution_salaire_femme']]
X = sm.add_constant(X)
y = modele1["Evolution_salaire"]
model = sm.OLS(y, X).fit()
model.summary()

### Analyse des Résultats de Régression OLS (Evolution des salaires)

#### Modèle de Régression OLS - Evolution des salaires

**Coefficients (coef) :**

- const (Intercept) : -0.0562
- Evolution_salaire_homme : 0.6353
- Evolution_salaire_femme : 0.3645

**P values (P>|t|) :**

- const (Intercept) : 0.311
- Evolution_salaire_homme : 0.001
- Evolution_salaire_femme : 0.007

- R-carré (R-squared) : 1.000

L'ajustement parfait du modèle indique que toutes les variations de la variable dépendante sont expliquées par les variables indépendantes.

- R-carré ajusté (Adj. R-squared) : 1.000

Il est également de 1.000, ce qui est cohérent avec le R-carré.

- Statistique F (F-statistic) : 6784.0

La statistique F teste l'hypothèse nulle selon laquelle tous les coefficients de pente sont égaux à zéro. Une valeur très élevée indique le rejet de l'hypothèse nulle.

- Prob (Omnibus) : et Prob (JB) :

Ces valeurs testent la normalité des résidus. Une p-value élevée suggère que les résidus suivent une distribution normale.

- Durbin-Watson : 0.799

La statistique Durbin-Watson teste l'autocorrélation des résidus. Une valeur proche de 2 est souhaitable, mais ici, elle est inférieure, indiquant une possible autocorrélation positive.

- Condition Number : 17.7

La condition number mesure la multicollinéarité. Une valeur basse indique une faible multicollinéarité.

**Commentaires sur les Résultats**

- Les coefficients significatifs (P>|t| < 0.05) indiquent que les variables indépendantes ont un impact significatif sur la variable dépendante.
- Cependant, le R-carré de 1.000 peut suggérer un surajustement du modèle, ce qui peut entraîner des problèmes lors de la généralisation à de nouveaux ensembles de données.
- La statistique Durbin-Watson inférieure à 2 suggère une possible autocorrélation positive des résidus.
- Les p values élevées pour les tests Omnibus et Jarque-Bera indiquent que les résidus peuvent ne pas suivre une distribution normale.

**En conclusion, bien que le modèle semble bien ajusté aux données d'entraînement, il est important de considérer la possibilité de surajustement et d'autres problèmes potentiels, notamment l'autocorrélation des résidus. Une exploration plus approfondie des données et une validation sur des ensembles de données distincts sont recommandées.**

<h2 style="padding:15px; color:#FFFF; background: #8a2be2">Conclusion sur l'Évolution Économique (2017-2023) et Prévisions Futures</h2>

1. **Evolution des salaires :**

- Une variation importante des salaires entre 2017 et 2023, avec des taux d'évolution variés pour les hommes et les femmes, ainsi que pour le salaire global. L'amélioration après 2020 suggère une reprise.

- Prédictions : Bien que divers facteurs économiques et sociaux puissent influencer les fluctuations, la tendance à la hausse devrait continuer.

2. **Evolution du taux de chômage :**

- Le taux de chômage fluctuera entre 2017 et 2023, avec des périodes de hausse et de baisse. Ces tendances peuvent être affectées par des changements sectoriels et régionaux.

- Prédictions : Les prévisions dépendent des secteurs d'emploi clés, de la politique gouvernementale et de l'économie mondiale.

3. **Les changements dans la consommation :**

- 2017 à 2023 : Les facteurs économiques et environnementaux peuvent entraîner des changements dans les habitudes de consommation, en particulier dans les secteurs alimentaire et énergétique.
- Prédictions : Les tendances du marché, la durabilité et les événements mondiaux peuvent influencer la consommation.

4. **L'inflation :**

- Le taux d'inflation varie entre 2017 et 2023, avec des périodes de stabilité et d'augmentation. Des facteurs tels que l'offre et la demande, les taux d'intérêt et les politiques gouvernementales peuvent avoir un impact sur l'inflation.

- Prédictions : Les indicateurs économiques mondiaux, les politiques monétaires et les événements géopolitiques doivent être pris en compte dans les prévisions.

5. **Les facteurs géopolitiques impliqués dans le conflit entre Israël et la Palestine :**

La stabilité économique mondiale, les prix de l'énergie et la confiance des consommateurs peuvent être affectés par des événements géopolitiques tels que le conflit Israël-Palestine.

**Prévisions Futures :**

Les prévisions économiques futures dépendent de nombreux facteurs, y compris la stabilité politique, les politiques gouvernementales, les tendances mondiales et les événements imprévus.
La vigilance face aux facteurs géopolitiques est essentielle pour anticiper les risques potentiels sur l'économie.

